In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
import os.path
from os import path
import seaborn as sns
from scipy import signal
from matplotlib.ticker import FormatStrFormatter

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [ ]:
### FUNCTION DEFINITION ###
## Read Dataframe ###


## 1, -- KDP_nu
KDP_nu_name=["CIR40" ,"CIR50"]
KDP_nu=[1,1.25]

## 2,  --  KDP_w_sat_1  -- -
KDP_w_sat_1_name=["0_33" ,"0_66" ,"1_00"]
KDP_w_sat_1=[0.33, 0.66 ,1.00]

## 3,  --  KDP_w_sat_2  -- -
KDP_w_sat_2_name=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov"]
KDP_w_sat_2=[380, 442 ,503]

## 4,  --  KDP_g_1  -- -
KDP_g_1_name=["0_33", "0_66" ,"1_00"]
KDP_g_1=[0.33, 0.66, 1.00]

## 5,  --  KDP_g_2  -- -
KDP_g_2_name=["ABW150Days" ,"ABW180Days"]
KDP_g_2=[150 ,180]

## 6,  --  KDP_c_p  -- -
KDP_c_p_name=["Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent"]
KDP_c_p=[1 ,1.5, 2]

KDP_lambda_name=["SD_RA","SD_RAR","SD_RARF"]
KDP_lambda=[0,0.1,0.2]

### CODE FOR OBTAINING DATE VALUES ####################
start ="2020-07-01"
end  = "2022-06-30"

start = datetime.datetime.strptime(start, "%Y-%m-%d")
end = datetime.datetime.strptime(end, "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]



for i in range(len(date_generated)):
    date_generated[i] = date_generated[i].strftime("%d-%b-%y")


def ReadDF(fname,colString):
    df_master = pd.DataFrame()
    dateOfPeak = []
    dayOfPeak = []
    numAtPeak = []
    vaccRate = []
    numRead = 0
    for i in range(len(KDP_nu)):
            for j in range(len(KDP_w_sat_1)):
                for k in range(len(KDP_w_sat_2)):
                    for l in range(len(KDP_g_1_name)):
                        for m in range(len(KDP_g_2)):
                            for n in range(len(KDP_c_p)):
                                for o in range(len(KDP_lambda)):
                                    if(path.exists(KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]+"/PopulationData/"+ fname)):
                                        df_temp = pd.read_csv(KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]+"/PopulationData/"+fname, delim_whitespace=True, header=1).transpose().loc[colString:]

                                        

                                        ExptId = KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]
                                        VaccineRt = str(KDP_c_p_name[n])
                                        CIR = str(KDP_nu_name[i]) 
                                        SD = str(KDP_lambda_name[o])
                                        KDPwsat = str(KDP_w_sat_2_name[k])
                                        KDPwsat1= str(KDP_w_sat_1_name[j])
                                        KDPg1=str(KDP_g_1_name[l])
                                        KDPg2=str(KDP_g_2_name[m])
                                        numRead+=1
                                        


                                        df_temp['ID'] = ExptId
                                        df_temp['Vaccine Rate'] = VaccineRt
                                        df_temp['CIR'] = CIR
                                        df_temp['SD'] = SD
                                        df_temp['KDPwsat2']=KDPwsat
                                        df_temp['KDPLambda']=KDP_lambda_name[o]
                                        df_temp['Scenario'] = numRead
                                        df_temp['KDPwsat1']=KDPwsat1
                                        df_temp['KDPg1']=KDPg1
                                        df_temp['KDPg2']=KDPg2
                                                                
                                        df_master = pd.concat([df_master,df_temp])


    print(str(numRead) + " samples have been read ")
    return df_master
                                    

In [ ]:
### COMPUTE ####
## Read Master Data ##



################# --- READ THE MASTER DATA ------------- ####################################

df_active= ReadDF("CovidPopulation.data","Total")




#### - Read the Nucleation Data
df_nucleation_total = ReadDF("CovidNucleation.data","Total")
df_nucleation_11 = ReadDF("CovidNucleation_11Yrs.data","Total")
df_nucleation_18 = ReadDF("CovidNucleation_18Yrs.data","Total")
df_nucleation_45 = ReadDF("CovidNucleation_45Yrs.data","Total")
df_nucleation_60 = ReadDF("CovidNucleation_60Yrs.data","Total")
df_nucleation_60plus = ReadDF("CovidNucleation_Above60Yrs.data","Total")


# ## Read the VaccinationData
# #### Read the Vaccination and the Antibody plots

df_antibody = ReadDF("Covid_Vaccinated.data","AntiBody")
df_Vaccinated = ReadDF("Covid_Antibody.data","AntiBody")




In [ ]:
### COMPUTE ####
## Nucleation DataFrames Ratio ##



############# HARDCODED ##########################
##  Total of 730 days simulated 
ROWS = 730

### getList of columns
listOfCols = list(df_nucleation_total.columns.values)

#### create a copy of all the dataframes to store the ratio values
df_nucleation_11_ratio = df_nucleation_11.copy(deep=True)
df_nucleation_18_ratio = df_nucleation_18.copy(deep=True)
df_nucleation_45_ratio = df_nucleation_45.copy(deep=True)
df_nucleation_60_ratio = df_nucleation_60.copy(deep=True)
df_nucleation_60plus_ratio = df_nucleation_60plus.copy(deep=True)

  ## Create just a copy of the dataframe

##### Compute Ratios ##################

for i in range(ROWS + 1):
    df_nucleation_11_ratio[[i]] =  df_nucleation_11_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_18_ratio[[i]] =  df_nucleation_18_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_45_ratio[[i]] =  df_nucleation_45_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_60_ratio[[i]] =  df_nucleation_60_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_60plus_ratio[[i]] =  df_nucleation_60plus_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 

df_nucleation_18plus_ratio = df_nucleation_45_ratio.copy(deep=True)
df_nucleation_18minus_ratio = df_nucleation_18_ratio.copy(deep=True)
## Append the total of 18+ dataframes to 
df_nucleation_18plus_ratio.iloc[:,0:731] += df_nucleation_60_ratio.iloc[:,0:731] + df_nucleation_60plus_ratio.iloc[:,0:731];

df_nucleation_18minus_ratio.iloc[:,0:731] +=df_nucleation_11_ratio.iloc[:,0:731]


## remove the NA values which occured due to division by zero
df_nucleation_11_ratio = df_nucleation_11_ratio.replace(np.nan,0)
df_nucleation_18_ratio = df_nucleation_18_ratio.replace(np.nan,0)
df_nucleation_45_ratio = df_nucleation_45_ratio.replace(np.nan,0)
df_nucleation_60_ratio = df_nucleation_60_ratio.replace(np.nan,0)
df_nucleation_60plus_ratio = df_nucleation_60plus_ratio.replace(np.nan,0)
df_nucleation_18plus_ratio = df_nucleation_18plus_ratio.replace(np.nan,0)




In [6]:
#####  FUNCTION DECLARATION   #########
#      Add PeakINfo                         ###

import scipy

def AddMeanInfo(df):
    meanval = []
    minval = []
    maxval = []
    for i in range(731):
        meanval.append(np.mean(df[i]))
        minval.append(np.min(df[i]))
        maxval.append(np.max(df[i]))
    df.append(minval)
    df.append(maxval)
    df.append(meanval)
    

### Compute the Peak in the signal #####
## 1, DataFrame
## 2, DateNUmber (0 to 730) to be assigned for NoPeak Scenario
## 3, NUmber of Peak value for No Peak
import scipy
def AddPeakInfo(df_master,df_nuc,noWaveDate,nowaveNumber,JiterNoPeak):
    dateOfPeakActive = []
    numAtPeakActive = []
    dateStringActive = [] 
    dateOfPeakNuc = []
    numAtPeakNuc = []
    dateStringNuc = [] 
    noiseDate = 0;
    noiseNum = 0;
    
    if(JiterNoPeak):    ## Add Jitter when Command is passed 
        rand=np.random.randn(1000) * 10+700
        rann=np.random.randn(1000) * 0.5+0.05
    noWave3=0
    for ia in range(len(df_master)):
        dopm,dictm = scipy.signal.find_peaks(df_master.iloc[[ia]].transpose().loc[400:730,'Total'],distance = 60,height=2000)
        dopn,dictn = scipy.signal.find_peaks(df_nuc.iloc[[ia]].transpose().loc[400:730,'Total'],distance = 60)
        if(len(dopm)>0 and len(dopn)>0):
            for ib in range(len(dopn)):
                if((abs(dopm[0]-dopn[ib]))<=15):
                    dateOfPeakActive.append(dopm[0]+400)
                    dateOfPeakNuc.append(dopn[ib]+400)
                    numAtPeakActive.append(df_active.iloc[ia,dopm[0]+400])
                    numAtPeakNuc.append(df_nucleation_total.iloc[ia,dopn[ib]+400])
                    break
                elif(ib==len(dopn)-1):
                    
                    dateOfPeakNuc.append(noWaveDate)
                    numAtPeakActive.append(df_active.iloc[ia,dopm[0]+400])
                    numAtPeakNuc.append(nowaveNumber)
                    dateOfPeakActive.append(dopm[0]+400)
        elif(len(dopm)>0 and len(dopn)==0):
            print(ia)
        else:
            dateOfPeakActive.append(noWaveDate)
            dateOfPeakNuc.append(noWaveDate)
            numAtPeakNuc.append(nowaveNumber)
            numAtPeakActive.append(nowaveNumber)
            noWave3=noWave3+1       
                    
            
  
                
    df_master['Date Of Peak'] = dateOfPeakActive
    df_master['Number At Peak'] = numAtPeakActive
    #df_master['DateString'] = dateString
    df_master['LogNumAtPeak'] = numpy.log10(numAtPeakActive)
    df_nuc['Date Of Peak'] = dateOfPeakNuc
    df_nuc['Number At Peak'] = numAtPeakNuc
    #df_master['DateString'] = dateString
    df_nuc['LogNumAtPeak'] = numpy.log10(numAtPeakNuc)
    print(noWave3)  
    return (df_master,df_nuc)
        

############ UPDATE FRACTION AT PEAK 
def fracAtPeak(df_plot):
#     df_plot=df[df['Date Of Peak']!=NOWAVEPEAKDATE]
    fracap = []
    for i in range(len(df_plot['Date Of Peak'])):
        if(df_plot['Date Of Peak'][i] != NOWAVEPEAKDATE):
            fracap.append(df_plot.iloc[i,df_plot['Date Of Peak'][i]])
        else:
            fracap.append(0)
    df_plot['fracAtDop'] = fracap
    return df_plot

    

In [9]:
###    COMPUTE            ####
## -PeakDataComputation - AgeGroup  ##

### -- PEAK DATA COMPUTATION --- #####
##########  UPDATE ALL THE NUCLEATION RATIOS AND DATE OF PEAKS FOR ALL DF;s ##############

import numpy


NOWAVEPEAKDATE =  670
NOWAVEPEAKNUMBER = 2


df_active,df_nucleation_total=AddPeakInfo(df_active,df_nucleation_total,NOWAVEPEAKDATE,NOWAVEPEAKNUMBER,False)


### Update the Peak NUcleation Cases on the Sub Distributions
listOfDF = [df_nucleation_11,df_nucleation_18,df_nucleation_45,df_nucleation_60,df_nucleation_60plus  , \
           df_nucleation_11_ratio,df_nucleation_18_ratio,df_nucleation_45_ratio,df_nucleation_60_ratio, \
            df_nucleation_60plus_ratio, df_nucleation_18plus_ratio ,df_nucleation_18minus_ratio]

# listOfDF = [df_nucleation_11_ratio,df_nucleation_18_ratio,df_nucleation_18plus_ratio]

#listOfDF = [df_nucleation_60]

for df in listOfDF:

    df['Date Of Peak']    =  df_nucleation_total['Date Of Peak'].tolist()

    df['Number At Peak']  = 0
    df['LogNumAtPeak']  = 0
    for row in range(df.shape[0]):
        Dop  = df_nucleation_total['Date Of Peak'][row]
        
        if(Dop != NOWAVEPEAKDATE ):   # Normal Date
#             if(df[Dop][row] == 0):
#                 val = 1
#                 df['LogNumAtPeak'][row] = numpy.log10(val)
#                 df['Number At Peak'][row] = df[Dop][row]+1
#             else:
#                 df['Number At Peak'][row] = df[Dop][row]+1
#                 df['LogNumAtPeak'][row] = numpy.log10(df[Dop][row]+1)
            df['Number At Peak'][row] = df[Dop][row]+2
            df['LogNumAtPeak'][row] = numpy.log10(df[Dop][row]+2)
            #if(df['CIR'][row]=='CIR50' and df['KDPwsat2'][row]=='ImmunEscp_Sep' and df['Vaccine Rate'][row]=='Vaccine_TwicePresent'):
             #    print(df[Dop][row], df['Number At Peak'][row], df['LogNumAtPeak'][row])
        else:
            df['Number At Peak'][row] = NOWAVEPEAKNUMBER
            df['LogNumAtPeak'][row] = numpy.log10(NOWAVEPEAKNUMBER)
            



### -- Compute the Fractional Data of the Time Series --- #####

# df_nucleation_11_ratio = fracAtPeak(df_nucleation_11_ratio)
# df_nucleation_18_ratio = fracAtPeak(df_nucleation_18_ratio)
# df_nucleation_45_ratio = fracAtPeak(df_nucleation_45_ratio)
# df_nucleation_60_ratio = fracAtPeak(df_nucleation_60_ratio)
# df_nucleation_60plus_ratio = fracAtPeak(df_nucleation_60plus_ratio)
# df_nucleation_18plus_ratio = fracAtPeak(df_nucleation_18plus_ratio)
# df_nucleation_18minus_ratio = fracAtPeak(df_nucleation_18minus_ratio)




# ###### Update the Ratio of Age groups data at peak to the master dataframe
# df_nucleation_total=df_nucleation_total[df_nucleation_total['Date Of Peak']!=NOWAVEPEAKDATE]
# df_nucleation_total['11FracAtPeak']=df_nucleation_11_ratio['fracAtDop'].tolist()
# df_nucleation_total['18FracAtPeak']=df_nucleation_18_ratio['fracAtDop'].tolist()
# df_nucleation_total['45FracAtPeak']=df_nucleation_45_ratio['fracAtDop'].tolist()
# df_nucleation_total['60FracAtPeak']=df_nucleation_60_ratio['fracAtDop'].tolist()
# df_nucleation_total['60plusFracAtPeak']=df_nucleation_60plus_ratio['fracAtDop'].tolist()
# df_nucleation_total['18plusFracAtPeak']=df_nucleation_18plus_ratio['fracAtDop'].tolist()
# df_nucleation_total['18minusFracAtPeak']=df_nucleation_18minus_ratio['fracAtDop'].tolist()
# df_nucleation_total.shape[0]

IndexError: positional indexers are out-of-bounds

In [ ]:
### ---- FUNCTION DEFINITION  ----   ####
### @@ DataGenerator - HistPlot   ###



# DATA GENERATOR - NUMBER OF CASES
####################################_____[[ DATA GENERERATOR ]] FOR NUMBER OF CASES ______________________ #################################

from scipy.stats import norm
import numpy
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.utils.fixes import parse_version
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt



def GenerateData(df_active,YColumn,Bandwidth,N_bins,LinePLotVals_Cases,LinePLotX_Cases,NoWaveRatio,boolHist):
    fig, ax = plt.subplots(3,2,figsize=(11,8))
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    kernel = 'epanechnikov'
    KDPCIR=["CIR= 40", "CIR= 50"]
    KDPImm=["EEscape Period - July","Escape Period - September", "Escape Period  - November"]
    legend = ["Present Rate","1.5x Present Rate","2x Present Rate"]
    
    
    
    for i in range(3):
        for j in range(2):
            df_kde=df_active.copy(deep=True)
            df_kde=df_kde[df_kde['CIR']==KDP_nu_name[j]]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            
            #index for storing Lineplot arrays
            plotIndex = 0
            
            for k in range(3):
                df_kdep = df_kde.copy(deep=True)
                df_kdep = df_kdep[df_kdep['Vaccine Rate']==KDP_c_p_name[k]].copy(deep=True)
                
                TotalRows = df_kdep.shape[0]   ### NUmber of Rows for this Selected Scenario
                
                ### Remove Rows With DATE_OF_PEAK = 670
                df_kdep = df_kdep[df_kdep['Date Of Peak']!=NOWAVEPEAKDATE].copy(deep=True)
                
                ### Remove the rows with NUmber at Peak < 500 
                df_kdep = df_kdep[df_kdep['Number At Peak']>200].copy(deep=True)
                
                print(df_active.shape[0])
#                 print("Scenario : ",KDPCIR[j],"_",KDPImm[i],"_",legend[k])
#                 print("Total Rows : ", TotalRows)
#                 print("No Wave Peeks : ", df_kdep.shape[0])
#                 print("Max Peak : ", max(df_kdep['Number At Peak']) )
                ## Truncated Rows
                TruncatedRows = df_kdep.shape[0]
                
                
                hist = sns.histplot(ax=ax[i,j],data=df_kdep,x=YColumn,stat='probability',\
                                    kde=True,label=legend[k],kde_kws={"bw_adjust":Bandwidth},
                                    color=colorVal[k],bins=N_bins)
                
 
                ratio = TruncatedRows/TotalRows    #### Compute Ratio of Wave Scenarios
                
                ratioNoWave = 1.0 - ratio;
                print("Total : " ,TotalRows, " TruncatedRows : ",TruncatedRows,   "  RatioNoWave : " , ratioNoWave)
                
                               
                if(len(hist.get_lines()) == plotIndex+1):
                    ll = hist.get_lines()[k].get_data()
                    LinePLotVals_Cases[i][j][k] = np.asarray(ll[1])
                    LinePLotX_Cases[i][j][k] = np.asarray(ll[0])

                    LinePLotVals_Cases[i][j][k] *=  ratio
                    NoWaveRatio[i][j][k]         =  ratioNoWave
                    
                    plotIndex += 1
                
                else:     ##### No Cases Available
                    print("SOMETHING WRONG")
                    NoWaveRatio[i][j][k]        =   ratioNoWave
                    LinePLotVals_Cases[i][j][k] =   np.zeros(len(LinePLotVals_Cases[i][j][k]))
                    LinePLotX_Cases[i][j][k]    =   np.asarray(hist.get_lines()[0].get_data()[0])
                
#                 ###GET MAX Y VALUE for Y lim
 #                maxYVal = max(LinePLotVals_Cases[i][j][k])
                
                if(not boolHist):
                    ax[i,j].containers[0].remove() # remove the bars
                else:
                    ax[i,j].legend(loc='upper right')
                    handles, labels = ax[i,j].get_legend_handles_labels()
                
                
                ax[i,j].set_title(KDPImm[i]+", "+ KDPCIR[j])
                ax[i,j].set_xlabel(YColumn)
    #             ax[i,j].set_xlim([1,5e6])
#                ax[i,j].set_ylim([0,maxYVal+0.05])
        
                
                

#     plt.legend(handles, labels, loc='lower center',title='Vaccination Rate',fancybox=True,ncol=3,bbox_to_anchor=(0.5,0.97))
    plt.tight_layout()

    plt.show()




In [ ]:
###--- FUNCTION DEFINITION ----     ####
### @@ CIR PLOTTER - NumCases   ###



###################### ____________________ PLOTTING SCRIPT _______________ ####################################
####  Input parameter -- DataFrame
####  Generates the 2*3 plot for Each CIR Scenario  [Num at Peak for 3 Immune Escape Variants] [Date of Peak for Immune Escape variants]

def CIR_plotter(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX, \
               NoWave_NumCases_Ratio,Nowave_DOP_Ratio,PlotNameExtension):
    print("CIOMES HERE")
    for cirv in range(2):
        ## 0 - cir40, 1-cir50
        CIRVARIANT = cirv;
        print("CIRV : ", cirv)
        ####### -- PLotter Parameters -- ######
        plt.rcParams["text.usetex"] = False
        plt.rcParams["font.family"] = "serif"
        plt.rcParams["font.serif"] = ["Computer Modern"]
        plt.rcParams['xtick.labelsize'] = 8
        plt.rcParams['ytick.labelsize'] = 8

        SMALL_SIZE = 8.5
        MEDIUM_SIZE = 8.5
        BIGGER_SIZE = 8.5

        plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        ## Set plotting parameters   -- Day of Peak
        markersizep = 2
        xtl = []
#         xt = [396,427,457,488,518,549,580,608,639,670+8]   ## 1st days of the months hardcoded 
#         xtl = ['   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','No Wave-3']
        xt = [396,457,518,580,639,670+8]   ## 1st days of the months hardcoded 
        xtl = ['   Aug\'21','   Oct\'21','Dec\'21','Feb\'22','Apr\'22','No Wave-3']
    #     for tickIndex in range(len(xt)):

    #         xtl.append(date_generated[xt[tickIndex]]);
    #         pass;


        ## Set plotting parameters   -- Number at Peak
        markersizep = 2
        nTicks_C = 10
        startDay_C = 0
        endDay_C = 4e6
#         xt_C = [0.3,1,2,3,4,5,6,7,8]
#         xtl_C = ["No Wave-3",r"         $10^{1}$",r"         $10^{2}$",r"    $10^{3}$",r"    $10^{4}$",r"    $10^{5}$",r"    $10^{6}$",r"    $10^{7}$",r"    $10^{8}$" ];
        xt_C = [0.3,2,4,6,8]
        xtl_C = ["No Wave-3",r"         $10^{2}$",r"    $10^{4}$",r"    $10^{6}$",r"    $10^{8}$" ];
   
        ## COMMON PLOTTING PARAMETERS
        markeveryVal = 15
        alphaVal = 0.3

        if(CIRVARIANT==0):
            y_limCol1_Val = 0.25
            y_limCol2_Val = 0.45
        else:
            y_limCol1_Val = 0.27
            y_limCol2_Val = 0.5      

        fig, ax = plt.subplots(3,2,figsize=(5,4),sharex=False,sharey=False)
        fig.tight_layout()

        colorVal = ['#7aa6c2','#4acf94','#b99bce']
        marker  = ['o','h','^']
        legend = ["Present Rate","1.5x Present Rate","2x Present Rate"]
        KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
        KDPCIR = ["CIR 40", "CIR 50"]
        
        ## MaxValueLegend 
        MaxValueLegend = [];
                 
        j=CIRVARIANT;
        col=0
        for i in range(3):
            for k in range(3):
        #             print( "(i,j) = (", str(i) ,"," ,str(j),") ",   " K val : " , str(k) )
        #            sns.lineplot(ax=ax[i,j],data=LinePLotVals[i][j][k],label=KDP_c_p_name[k])
                ax[i,col].plot(LinePLotX[i][j][k],LinePLotVals[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)

                ax[i,col].fill_between(LinePLotX[i][j][k],LinePLotVals[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX = LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[lenX - 1]
                freq                               =  Nowave_DOP_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[lenX - 1]                =  NOWAVEPEAKDATE + k*8
                BarData[lenX - 1]                  =  freq   
                         
                ## BAr Plot
                ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=8)
                
                LinePLotX[i][j][k][lenX - 1]      = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                
                
                
            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(KDPImm[i] +", "+ KDPCIR[j], y=10.0, pad=-14)
            ax[i,col].text(.48,.82,KDPImm[i] +", "+ KDPCIR[j],horizontalalignment='center',transform=ax[i,col].transAxes)
            ax[i,col].set_ylabel("""Probability of D""")
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
#             ax[i,col].set_xlim()
            ax[i,col].set_ylim((0,max(MaxValueLegend)+0.1))
            ax[i,col].set_xticks(xt);
            ax[i,col].set_xticklabels([]);
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.18,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

        ax[2,col].set_xlabel("""Date of peak (D)""")
        ax[2,col].set_xticks(xt);
        ax[2,col].set_xticklabels(xtl);
        ax[2,col].tick_params(axis='x', rotation=60);


        ### --- NUMBER OF CASES ----- ######

        col=1
                 
        MaxValueLegend = []

        for i in range(3):
            for k in range(3):
                ax[i,col].plot(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)
                ax[i,col].fill_between(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX                          =  LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[0]
                freq                               =  NoWave_NumCases_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[0]                       =  0 + k*0.2 + 0.1
                BarData[0]                         =  freq   
                
                
                
                
                ## BAr Plot
                ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=0.2)
                
                LinePLotX[i][j][k][0] = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                

            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(, position=(0.5, 4.6))
            ax[i,col].text(.48,.85,KDPImm[i] +", "+ KDPCIR[j],horizontalalignment='center',transform=ax[i,col].transAxes)
    #         ax[i,col].set_ylabel("""1""")
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
            ax[i,col].set_xticks(xt_C);
            ax[i,col].set_xlim([0,8]);
            ax[i,col].set_xticklabels([]);
            ax[i,col].set_ylim((0,max(MaxValueLegend)+0.1))
            ax[i,col].set_ylabel("""Probability of A""")
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.18,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        
        if(PlotNameExtension == "TotalActive"):
            ax[2,col].set_xlabel("""Total active cases at peak (A)""")
        else:
            ax[2,col].set_xlabel("""Daily confirmed cases at peak (A)""")
    #     ax[2,col].yaxis.set_label_coords(-0.075,.5)
        ax[2,col].set_xticks(xt_C);
        ax[2,col].set_xticklabels(xtl_C);
        ax[2,col].tick_params(axis='x', rotation=60);



        fig.legend(handles, labels, loc='lower center',title='',ncol=3,bbox_to_anchor=(0.5,0.98),frameon=False)
        plt.tight_layout()
        
        print("SavePlot Start");
        if(CIRVARIANT==0): 
            name = "CIR40"
            name = name + "_" + PlotNameExtension 
            plt.savefig(name + ".png",dpi=300,bbox_inches='tight')
        else:
            print(" SHOULD COME HERE")
            name = "CIR50"
            name = name + "_" + PlotNameExtension
            plt.savefig(name + ".png",dpi=300,bbox_inches='tight')
        
        print("SavePlot End");
        
        
        plt.show()
        plt.draw()



In [ ]:
###--- FUNCTION DEFINITION ----     ####
### @@ CIR PLOTTER - NumCases   ###



###################### ____________________ PLOTTING SCRIPT _______________ ####################################
####  Input parameter -- DataFrame
####  Generates the 2*3 plot for Each CIR Scenario  [Num at Peak for 3 Immune Escape Variants] [Date of Peak for Immune Escape variants]

def CIR_plotter(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX, \
               NoWave_NumCases_Ratio,Nowave_DOP_Ratio,PlotNameExtension):
    print("CIOMES HERE")
    for cirv in range(2):
        ## 0 - cir40, 1-cir50
        CIRVARIANT = cirv;
        print("CIRV : ", cirv)
        ####### -- PLotter Parameters -- ######
        plt.rcParams["text.usetex"] = False
        plt.rcParams["font.family"] = "serif"
        plt.rcParams["font.serif"] = ["Computer Modern"]
        plt.rcParams['xtick.labelsize'] = 8
        plt.rcParams['ytick.labelsize'] = 8

        SMALL_SIZE = 16
        MEDIUM_SIZE = 16
        BIGGER_SIZE = 16

        plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        ## Set plotting parameters   -- Day of Peak
        markersizep = 5
        xtl = []
#         xt = [396,427,457,488,518,549,580,608,639,670+8]   ## 1st days of the months hardcoded 
#         xtl = ['   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','No Wave-3']
        xt = [396,427,457,488,518,549,580,608,639]   ## 1st days of the months hardcoded 
        xtl = ['   Aug\'21','','   Oct\'21','','Dec\'21','','Feb\'22','','Apr\'22']
    #     for tickIndex in range(len(xt)):

    #         xtl.append(date_generated[xt[tickIndex]]);
    #         pass;


        ## Set plotting parameters   -- Number at Peak
        markersizep = 5
        nTicks_C = 10
        startDay_C = 0
        endDay_C = 4e6
#         xt_C = [0.3,1,2,3,4,5,6,7,8]
#         xtl_C = ["No Wave-3",r"         $10^{1}$",r"         $10^{2}$",r"    $10^{3}$",r"    $10^{4}$",r"    $10^{5}$",r"    $10^{6}$",r"    $10^{7}$",r"    $10^{8}$" ];
        xt_C = [2,3,4,5,6,7]#,8]
        xtl_C = [r"         $10^{2}$","",r"    $10^{4}$","",r"    $10^{6}$",""]#,r"    $10^{8}$" ];
   
        ## COMMON PLOTTING PARAMETERS
        markeveryVal = 15
        alphaVal = 0.3

        if(CIRVARIANT==0):
            y_limCol1_Val = 0.35
            y_limCol2_Val = 0.25
        else:
            y_limCol1_Val = 0.27
            y_limCol2_Val = 0.5      

        fig, ax = plt.subplots(3,2,figsize=(11,8),sharex=False,sharey=False)
        #fig.tight_layout()

        colorVal = ['#7aa6c2','#4acf94','#b99bce']
        marker  = ['o','h','^']
        legend = ["VR-100P","VR-150P","VR-200P"]
        KDPImm = ["IENV-Jul21", "IENV-Sep21","IENV-Nov21" ]
        KDPCIR = ["CIR40", "CIR50"]
        
        ## MaxValueLegend 
        MaxValueLegend = [];
                 
        j=CIRVARIANT;
        col=0
        for i in range(3):
            for k in range(3):
        #             print( "(i,j) = (", str(i) ,"," ,str(j),") ",   " K val : " , str(k) )
        #            sns.lineplot(ax=ax[i,j],data=LinePLotVals[i][j][k],label=KDP_c_p_name[k])
                ax[i,col].plot(LinePLotX[i][j][k],LinePLotVals[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)

                ax[i,col].fill_between(LinePLotX[i][j][k],LinePLotVals[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX = LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[lenX - 1]
                freq                               =  Nowave_DOP_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[lenX - 1]                =  NOWAVEPEAKDATE + k*8
                BarData[lenX - 1]                  =  freq   
                         
                ## BAr Plot
                #ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=8)
                
                LinePLotX[i][j][k][lenX - 1]      = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                
                
                
            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(KDPImm[i] +", "+ KDPCIR[j], y=10.0, pad=-14)
            ax[i,col].text(.88,.82,KDPImm[i],horizontalalignment='right',transform=ax[i,col].transAxes)
            ax[i,col].set_ylabel("""P(D)""")
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
#             ax[i,col].set_xlim()
            ax[i,col].set_ylim((0,y_limCol1_Val))
            ax[i,col].set_xticks(xt);
            ax[i,col].set_xticklabels([]);
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.18,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

        ax[2,col].set_xlabel("""Date of peak (D)""")
        ax[2,col].set_xticks(xt);
        ax[2,col].set_xticklabels(xtl);
        ax[2,col].tick_params(axis='x', rotation=30);


        ### --- NUMBER OF CASES ----- ######

        col=1
                 
        MaxValueLegend = []

        for i in range(3):
            for k in range(3):
                ax[i,col].plot(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)
                ax[i,col].fill_between(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX                          =  LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[0]
                freq                               =  NoWave_NumCases_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[0]                       =  0 + k*0.2 + 0.1
                BarData[0]                         =  freq   
                
                
                
                
                ## BAr Plot
                #ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=0.2)
                
                LinePLotX[i][j][k][0] = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                

            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(, position=(0.5, 4.6))
            ax[i,col].text(.88,.85,KDPImm[i],horizontalalignment='right',transform=ax[i,col].transAxes)
    #         ax[i,col].set_ylabel("""1""")
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
            ax[i,col].set_xticks(xt_C);
            ax[i,col].set_xlim([2,7]);
            ax[i,col].set_xticklabels([]);
            ax[i,col].set_ylim((0,y_limCol2_Val))
            ax[i,col].set_ylabel("""P(C)""")
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.15,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        
        if(PlotNameExtension == "TotalActive"):
            ax[2,col].set_xlabel("""Total active cases at peak (A)""")
        else:
            ax[2,col].set_xlabel("""Daily confirmed cases at peak (C)""")
    #     ax[2,col].yaxis.set_label_coords(-0.075,.5)
        ax[2,col].set_xticks(xt_C);
        ax[2,col].set_xticklabels(xtl_C);
        ax[2,col].tick_params(axis='x', rotation=30);

        ph = [plt.plot([],marker="", ls="")[0]] # Canvas
        handles = ph + handles
        labels= ["Vaccination Rate:"]+labels
        fig.legend(handles, labels, loc=8,title='',ncol=4,bbox_to_anchor=(0.5,0.98),frameon=False,title_fontsize=16,fontsize=16)
        plt.tight_layout()
        fig.subplots_adjust(wspace=0.3,hspace=0.3)
        print("SavePlot Start");
        if(CIRVARIANT==0): 
            name = "CIR40"
            name = name + "_" + PlotNameExtension 
            plt.savefig(name + ".png",dpi=300,bbox_inches='tight')
        else:
            print(" SHOULD COME HERE")
            name = "CIR50"
            name = name + "_" + PlotNameExtension
            plt.savefig(name + ".png",dpi=300,bbox_inches='tight')
        
        print("SavePlot End");
        
        
        plt.show()
        plt.draw()



In [ ]:
!pip install libtiff

In [7]:
###-FUNCTION DEFINITION --3x3 Divij--     ####
### @@ CIR PLOTTER - NumCases   ###


from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits import axisartist
###################### ____________________ PLOTTING SCRIPT _______________ ####################################
####  Input parameter -- DataFrame
####  Generates the 2*3 plot for Each CIR Scenario  [Num at Peak for 3 Immune Escape Variants] [Date of Peak for Immune Escape variants]
from matplotlib import axes
from PIL import Image
import io
def CIR_plotter3x3(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX,LinePLotVals_Con,LinePLotX_Con, \
               NoWave_NumCases_Ratio,Nowave_DOP_Ratio,NoWave_NumCases_Ratio_Con,PlotNameExtension): 
    print("COMES HERE")
    for cirv in range(2):
        ## 0 - cir40, 1-cir50
        CIRVARIANT = cirv;
        print("CIRV : ", cirv)
        ####### -- PLotter Parameters -- ######
        plt.rcParams["text.usetex"] = False
        plt.rcParams["font.family"] = "serif"
        plt.rcParams["font.serif"] = ["Computer Modern"]
        plt.rcParams['xtick.labelsize'] = 8
        plt.rcParams['ytick.labelsize'] = 8

        SMALL_SIZE = 14
        MEDIUM_SIZE = 14
        BIGGER_SIZE = 14

        plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        ## Set plotting parameters   -- Day of Peak
        markersizep = 5
        xtl = []
#         xt = [396,427,457,488,518,549,580,608,639,670+8]   ## 1st days of the months hardcoded 
#         xtl = ['   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','No Wave-3']
        xt = [396,427,457,488,518,549,580,608,639]   ## 1st days of the months hardcoded 
        xtl = ['    Aug\'21','','    Oct\'21','','Dec\'21','','Feb\'22','','Apr\'22']
    #     for tickIndex in range(len(xt)):

    #         xtl.append(date_generated[xt[tickIndex]]);
    #         pass;


        ## Set plotting parameters   -- Number at Peak
        markersizep = 5
        nTicks_C = 10
        startDay_C = 0
        endDay_C = 4e6
#         xt_C = [0.3,1,2,3,4,5,6,7,8]
#         xtl_C = ["No Wave-3",r"         $10^{1}$",r"         $10^{2}$",r"    $10^{3}$",r"    $10^{4}$",r"    $10^{5}$",r"    $10^{6}$",r"    $10^{7}$",r"    $10^{8}$" ];
        xt_C = [2,3,4,5,6,7]#,8]
        xtl_C = [r"        $10^{2}$","",r"   $10^{4}$","",r"    $10^{6}$",""]#,r"    $10^{8}$" ];
   
        ## COMMON PLOTTING PARAMETERS
        markeveryVal = 15
        alphaVal = 0.3

        if(CIRVARIANT==0):
            y_limCol1_Val = 0.35
            y_limCol2_Val = 0.25
            y_limCol3_Val = 0.25
        else:
            y_limCol1_Val = 0.35
            y_limCol2_Val = 0.25
            y_limCol3_Val = 0.25

        fig, ax = plt.subplots(3,2,figsize=(11,8),sharex=False,sharey=False)
        fig.tight_layout()
        
        colorVal = ['#7aa6c2','#4acf94','#b99bce']
        marker  = ['o','h','^']
        legend = ["VR-100P","VR-150P","VR-200P"]
        KDPImm = ["IENV-Jul21", "IENV-Sep21","IENV-Nov21" ]
        subfig=["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)","(i)","(j)","(k)","(l)","(m)","(n)","(o)"]
        KDPCIR = ["CIR40", "CIR50"]
        sfg=0
        ## MaxValueLegend 
        MaxValueLegend = [];
                 
        j=CIRVARIANT;
        col=0
        for i in range(3):
            for k in range(3):
        #             print( "(i,j) = (", str(i) ,"," ,str(j),") ",   " K val : " , str(k) )
        #            sns.lineplot(ax=ax[i,j],data=LinePLotVals[i][j][k],label=KDP_c_p_name[k])
                ax[i,col].plot(LinePLotX[i][j][k],LinePLotVals[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)

                ax[i,col].fill_between(LinePLotX[i][j][k],LinePLotVals[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX = LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[lenX - 1]
                freq                               =  Nowave_DOP_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[lenX - 1]                =  NOWAVEPEAKDATE + k*8
                BarData[lenX - 1]                  =  freq   
                         
                ## BAr Plot
                #ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=8)
                
                LinePLotX[i][j][k][lenX - 1]      = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                
                
                
            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(KDPImm[i] +", "+ KDPCIR[j], y=10.0, pad=-14)
            ax[i,col].text(.88,.82,subfig[sfg],horizontalalignment='right',transform=ax[i,col].transAxes)
            sfg+=1
            ax[i,col].set_ylabel(KDPImm[i]+"""\nP(D)""")
            
            
            
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
#             ax[i,col].set_xlim()
            ax[i,col].set_ylim((0,y_limCol1_Val))
            ax[i,col].set_xticks(xt);
            ax[i,col].set_xticklabels([]);
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.16,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

        ax[2,col].set_xlabel("""Date of peak (D)""")
        ax[2,col].set_xticks(xt);
        ax[2,col].set_xticklabels(xtl);
        ax[2,col].tick_params(axis='x', rotation=30);


        ### --- NUMBER OF CASES ----- ######

        col=1
                 
        MaxValueLegend = []

        for i in range(3):
            for k in range(3):
                ax[i,col].plot(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)
                ax[i,col].fill_between(LinePLotX_Cases[i][j][k],LinePLotVals_Cases[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                
                MaxValueLegend.append(max(LinePLotVals[i][j][k]))
                
                BarChartX                          =  LinePLotX[i][j][k].copy()
                
                ##Generate Bar for no Wave Scenario
                # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
                # Generate a data with zeros, and append the last value of data as Prob for no peak
                lenX                               =  len(BarChartX)
                PrevXVal                           =  BarChartX[0]
                freq                               =  NoWave_NumCases_Ratio[i][j][k]
                BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
                BarChartX[0]                       =  0 + k*0.2 + 0.1
                BarData[0]                         =  freq   
                
                
                
                
                ## BAr Plot
                #ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=0.2)
                
                LinePLotX[i][j][k][0] = PrevXVal
                
                #Append MaxValue to legend Array
                MaxValueLegend.append(freq)
                
            
            ax[i,col].legend(title='Vaccination Rate',loc='upper right')
    #         ax[i,col].set_title(, position=(0.5, 4.6))
            ax[i,col].text(.88,.85,subfig[sfg],horizontalalignment='right',transform=ax[i,col].transAxes)
            sfg+=1
    #         ax[i,col].set_ylabel("""1""")
            ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
            ax[i,col].set_xticks(xt_C);
            ax[i,col].set_xlim([3,7]);
            ax[i,col].set_xticklabels([]);
            ax[i,col].set_ylim((0,y_limCol2_Val))
            ax[i,col].set_ylabel("""P(A)""")
            handles, labels = ax[i,col].get_legend_handles_labels()
            ax[i,col].get_legend().remove()
            ax[i,col].get_yaxis().set_label_coords(-0.16,0.5)
            ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
            

        
        if(PlotNameExtension == "TotalActive"):
            ax[2,col].set_xlabel("""Total active cases at peak (A)""")
        else:
            ax[2,col].set_xlabel("""Daily active cases at peak (A)\n\n\n\nDaily confirmed cases at peak (C)""")
    #     ax[2,col].yaxis.set_label_coords(-0.075,.5)
        xt_C1 = [3,4,5,6,7]#,8]
        xtl_C1 = [r"$10^{3}$","",r"$10^{5}$","",r"$10^{7}$"]#,r"    $10^{8}$" ];
        ax[2,col].set_xticks(xt_C1);
        ax[2,col].set_xticklabels(xtl_C1);
        ax[2,col].tick_params(axis='x', rotation=0);
        
        
        ax3=ax[2,col].twiny()
        ax3.spines["top"].set_position(("axes", -0.57))
        #ax3.axis["bottom"].toggle(all=True)
#             ax3.spines.bottom.set_position(("axes", 1.2))
#            ax3.set_xticks(xt_C);
        ax3.set_xlim([2,6]);
        ax3.set_xticklabels([])
        
            
        
        
        #ax3.set_xlabel("""Daily confirmed cases at peak (C)""")
    #     ax[2,col].yaxis.set_label_coords(-0.075,.5)
        xt_C2 = [2,3,4,5,6]#,8]
        xtl_C2 = [r"$10^{2}$","",r"$10^{4}$","",r"$10^{6}$"]#,r"    $10^{8}$" ];
        ax3.set_xticks(xt_C2);
        ax3.set_xticklabels(xtl_C2);
        ax3.tick_params(axis='x', rotation=0);
        
        
        
        #ax3.xaxis.tick_bottom()
        
        #make_patch_spines_invisible(ax3)
# Second, show the right spine.
        #ax3.spines["bottom"].set_visible(True)
        #####Daily confirmed Cases#########
        
#         col=2
                 
#         MaxValueLegend = []

#         for i in range(3):
#             for k in range(3):
#                 ax[i,col].plot(LinePLotX_Con[i][j][k],LinePLotVals_Con[i][j][k],label = legend[k],linewidth=0.75,color=colorVal[k],marker=marker[k],markersize=markersizep,markevery=markeveryVal)
#                 ax[i,col].fill_between(LinePLotX_Con[i][j][k],LinePLotVals_Con[i][j][k],0, alpha=alphaVal, color=colorVal[k])
                
#                 MaxValueLegend.append(max(LinePLotVals_Con[i][j][k]))
                
#                 BarChartX                          =  LinePLotX[i][j][k].copy()
                
#                 ##Generate Bar for no Wave Scenario
#                 # Get the current X array and then append last x as WAVEPEAKDATE  ( x Value )
#                 # Generate a data with zeros, and append the last value of data as Prob for no peak
#                 lenX                               =  len(BarChartX)
#                 PrevXVal                           =  BarChartX[0]
#                 freq                               =  NoWave_NumCases_Ratio_Con[i][j][k]
#                 BarData                            =  np.full(shape=len(BarChartX), fill_value=0.0,dtype=np.float)
#                 BarChartX[0]                       =  0 + k*0.2 + 0.1
#                 BarData[0]                         =  freq   
                
                
                
                
#                 ## BAr Plot
#                 #ax[i,col].bar(BarChartX,BarData, color=colorVal[k],width=0.2)
                
#                 LinePLotX[i][j][k][0] = PrevXVal
                
#                 #Append MaxValue to legend Array
#                 MaxValueLegend.append(freq)
                

#             ax[i,col].legend(title='Vaccination Rate',loc='upper right')
#     #         ax[i,col].set_title(, position=(0.5, 4.6))
#             ax[i,col].text(.88,.85,subfig[sfg],horizontalalignment='right',transform=ax[i,col].transAxes)
#             sfg+=1
#     #         ax[i,col].set_ylabel("""1""")
#             ax[i,col].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
#             ax[i,col].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
#             ax[i,col].set_xticks(xt_C);
#             ax[i,col].set_xlim([2,6]);
#             ax[i,col].set_xticklabels([])
            
#             ax[i,col].set_ylim((0,y_limCol3_Val))
#             ax[i,col].set_ylabel("""P(C)""")
            
#             ax2=ax[i,col].twinx()
#             ax2.set_ylabel(KDPImm[i])
#             ax2.tick_params(axis='y',which='both',right=False,labelright=False)
            
            
            
#             handles, labels = ax[i,col].get_legend_handles_labels()
#             ax[i,col].get_legend().remove()
#             ax[i,col].get_yaxis().set_label_coords(-0.18,0.5)
#             ax[i,col].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        
#         if(PlotNameExtension == "TotalActive"):
#             ax[2,col].set_xlabel("""Total active cases at peak (A)""")
#         else:
#             ax[2,col].set_xlabel("""Daily confirmed cases at peak (C)""")
#     #     ax[2,col].yaxis.set_label_coords(-0.075,.5)
#         xt_C2 = [2,3,4,5,6]#,8]
#         xtl_C2 = [r"          $10^{2}$","",r"     $10^{4}$","",r"     $10^{6}$"]#,r"    $10^{8}$" ];
#         ax[2,col].set_xticks(xt_C2);
#         ax[2,col].set_xticklabels(xtl_C2);
#         ax[2,col].tick_params(axis='x', rotation=30);
        
################################################################################################################
        ax4=ax[0,1].twinx()
        ax4.set_ylabel("\nP(C)")
        
        ax4.set_ylim((0,y_limCol2_Val))
        ax4.set_yticklabels(["","",""]);
        ax4.tick_params(labelright=False)
        ax4.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax4.get_yaxis().set_label_coords(1,0.5)
        ax5=ax[1,1].twinx()
        ax5.set_ylabel("\nP(C)")
       
        ax5.set_ylim((0,y_limCol2_Val))
        ax5.set_yticklabels(["","",""]);
        ax5.tick_params(labelright=False)
        ax5.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax5.get_yaxis().set_label_coords(1,0.5)
        ax6=ax[2,1].twinx()
        ax6.set_ylabel("\nP(C)")
        
        ax6.set_ylim((0,y_limCol2_Val))
        ax6.set_yticklabels(["","",""]);
        ax6.tick_params(labelright=False)
        ax6.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax6.get_yaxis().set_label_coords(1,0.5)
        ph = [plt.plot([],marker="", ls="")[0]] # Canvas
        handles = ph + handles
        labels= ["Vaccination Rate:"]+labels
        fig.legend(handles, labels, loc=10,title='',ncol=4,bbox_to_anchor=(0.475,0.98),frameon=False,title_fontsize=15,fontsize=15)
        
        fig.tight_layout()
        fig.subplots_adjust(wspace=0.3,hspace=0.3,bottom=0.2)
        
        
        print("SavePlot Start");
        if(CIRVARIANT==0): 
            name = "CIR40"
            name = name + "_" + PlotNameExtension+"divij" 
            
            plt.savefig(name + ".png",dpi=500,bbox_inches='tight')
            png1=io.BytesIO()
            plt.savefig(png1,format="png",dpi=500,bbox_inches='tight')
            png2=Image.open(png1)
            png2.save(name+".tiff",compression='tiff_lzw')
            png1.close()
#             fig.savefig(name+".tiff",dpi=500,bbox_inches='tight')
        else:
            print(" SHOULD COME HERE")
            name = "CIR50"
            name = name + "_" + PlotNameExtension+"divij"
            plt.savefig(name + ".png",dpi=300,bbox_inches='tight')
        
        print("SavePlot End");
        
        
        plt.show()
        plt.draw()



In [1]:
###--- PLOTTING CELL   3x3 Divij   ####
### @@ CIR PLOT    ###

############### Function for Plotting #####################################

def plotDataFrame3x3(DataFrameName,DataFrameName2,plotName):
#     DataFrameName = AddPeakInfo(DataFrameName,700,False)

    ## For Number of Cases    
    LinePLotVals_Cases = np.empty(shape=(3,2,3,200))
    LinePLotX_Cases = np.empty(shape=(3,2,3,200))

    ## For Date of Peak
    LinePLotVals = np.empty(shape=(3,2,3,200))
    LinePLotX = np.empty(shape=(3,2,3,200))
    
    ##For daily confirmed cases
    LinePLotVals_Con = np.empty(shape=(3,2,3,200))
    LinePLotX_Con = np.empty(shape=(3,2,3,200))
    
    
    ### For Storing Ratio of No Peak in Each cases
    NoWave_NumCases_Ratio = np.empty(shape = (3,2,3,1))
    Nowave_DOP_Ratio = np.empty(shape = (3,2,3,1))
    NoWave_NumCases_Ratio_Con = np.empty(shape = (3,2,3,1))
    
    
    ### Data for Number at Peak
    print("Size before GenData 1: ", DataFrameName.shape[0])
    GenerateData(DataFrameName,'LogNumAtPeak',0.4,10,LinePLotVals_Cases,LinePLotX_Cases,NoWave_NumCases_Ratio,True)
    print("Size Aftr GenData 1: ", df_nucleation_total.shape[0])
    
    ### Data for Day of Peak
    print("Size before GenData1: ", DataFrameName.shape[0])
    GenerateData(DataFrameName,'Date Of Peak',0.4,10,LinePLotVals,LinePLotX,Nowave_DOP_Ratio,True)
    print("Size After GenData1: ", DataFrameName.shape[0])
    
    #####Data for daily confirmed
    print("Size before GenData1: ", DataFrameName2.shape[0])
    GenerateData(DataFrameName2,'LogNumAtPeak',0.4,10,LinePLotVals_Con,LinePLotX_Con,NoWave_NumCases_Ratio_Con,True)
    print("Size After GenData1: ", DataFrameName2.shape[0])
    ### PLot the values in seperate plots
    CIR_plotter3x3(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX,LinePLotVals_Con,LinePLotX_Con,\
                NoWave_NumCases_Ratio,Nowave_DOP_Ratio,NoWave_NumCases_Ratio_Con,plotName)
    
#     SCATTER_plotter(df_active,plotName)

# plotDataFrame(df_nucleation_11,"nucleation_11")
# plotDataFrame(df_nucleation_18,"nucleation_18")
# plotDataFrame(df_nucleation_45,"nucleation_45")
# plotDataFrame(df_nucleation_60,"nucleation_60")
# plotDataFrame(df_nucleation_60plus,"nucleation_60plus")
#plotDataFrame(df_active,"TotalActive")
plotDataFrame3x3(df_active,df_nucleation_total,"Divij")
#plotDataFrame(df_nucleation_total,"DailyConfirmed")



# #EXTRA -- REMOVE THEM
# LinePLotVals_Cases = np.empty(shape=(3,2,3,200))
# LinePLotX_Cases = np.empty(shape=(3,2,3,200))

# ## For Date of Peak
# LinePLotVals = np.empty(shape=(3,2,3,200))
# LinePLotX = np.empty(shape=(3,2,3,200))


# ### For Storing Ratio of No Peak in Each cases
# NoWave_NumCases_Ratio = np.empty(shape = (3,2,3,1))
# Nowave_DOP_Ratio = np.empty(shape = (3,2,3,1))

# HANDLES,LABELS = CIR_plotter(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX, \
#                 NoWave_NumCases_Ratio,Nowave_DOP_Ratio,"test")

NameError: name 'df_active' is not defined

In [ ]:
###--- PLOTTING CELL ----     ####
### @@ CIR PLOT    ###
############### Function for Plotting #####################################
noWaveRatioArray = np.empty(shape = (3,2,3,1))
def plotDataFrame(DataFrameName,plotName):
#     DataFrameName = AddPeakInfo(DataFrameName,700,False)
    ## For Number of Cases    
    LinePLotVals_Cases = np.empty(shape=(3,2,3,200))
    LinePLotX_Cases = np.empty(shape=(3,2,3,200))
    ## For Date of Peak
    LinePLotVals = np.empty(shape=(3,2,3,200))
    LinePLotX = np.empty(shape=(3,2,3,200))
    
    
    ### For Storing Ratio of No Peak in Each cases
    NoWave_NumCases_Ratio = np.empty(shape = (3,2,3,1),dtype=np.float)
    Nowave_DOP_Ratio = np.empty(shape = (3,2,3,1),dtype=np.float)
    
    
    ### Data for Number at Peak
    GenerateData(DataFrameName,'LogNumAtPeak',0.4,10,LinePLotVals_Cases,LinePLotX_Cases,NoWave_NumCases_Ratio,True)
    
    ansArray = np.empty(shape=(3,3),dtype=np.float)
    
    
    def createNowave(NoWave_NumCases_Ratio,ansArray):
        for i in range(3):
            for j in range(3):
                ansArray[i][j] = NoWave_NumCases_Ratio[i][0][j]
        return ansArray
    createNowave(NoWave_NumCases_Ratio,ansArray)
    print(ansArray)
    ### Data for Day of Peak
    GenerateData(DataFrameName,'Date Of Peak',0.4,10,LinePLotVals,LinePLotX,Nowave_DOP_Ratio,True)
    
    ### PLot the values in seperate plots
    CIR_plotter(LinePLotVals_Cases,LinePLotX_Cases,LinePLotVals,LinePLotX, \
                NoWave_NumCases_Ratio,Nowave_DOP_Ratio,plotName)
    
#     SCATTER_plotter(df_active,plotName)
# plotDataFrame(df_nucleation_11,"nucleation_11")
# plotDataFrame(df_nucleation_18,"nucleation_18")
# plotDataFrame(df_nucleation_45,"nucleation_45")
# plotDataFrame(df_nucleation_60,"nucleation_60")
#plotDataFrame(df_nucleation_60plus,"nucleation_60plus")
plotDataFrame(df_nucleation_18plus,"18plus")
# data = plotDataFrame(df_active,"TotalActive")
noRatio = plotDataFrame(df_active,"")


print(ansArray)


In [ ]:
###--- FUNCTION DEFINITION ----     ####
### @@ SCATTER PLOTTER - NumCases   ###



###################### _____________________ SCATTER PLOTTTER ______________________________ ##############################
        
import warnings
warnings.filterwarnings('ignore')


def SCATTER_plotter(df_active,plotName):
    
    from scipy.stats import norm
    import numpy
    import sklearn
    from sklearn.neighbors import KernelDensity
    from sklearn.utils.fixes import parse_version
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import LeaveOneOut
    from sklearn import preprocessing
    import matplotlib.patches as mpatches
    fig, ax = plt.subplots(3,2,figsize=(11,8),sharex=True,sharey=True)

    ####### -- PLotter Parameters -- ######
    plt.rcParams["text.usetex"] = False
    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.serif"] = ["Computer Modern"]
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    plt.rcParams["font.serif"]
    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    xtl = []
    xt = [396,427,457,488,518,549,580,608,639,670]   ## 1st days of the months hardcoded 
    xtl = ['   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','No wave-3']



    ## Set plotting parameters   -- Number at Peak
    markersizep = 1.9
    nTicks_C = 10
    startDay_C = 0
    endDay_C = 4e6
    xt_C = [1e-2,1e0,1e2,1e4,1e6,1e8,1e9]
    xtl_C = ["0",r"$10^{0}$",r"$10^{2}$",r"$10^{4}$",r"$10^{6}$",r"$10^{8}$",r"$10^{9}$"];


    #### PLotting Colors and other Parameters 
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    kernel = 'epanechnikov'
    KDPCIR=["CIR 40", "CIR 50"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    JitterDate = 0.01
    JitterNum = 0.00000007
        #LinePLotVals = np.empty(shape=(4,4,4,200))
    #LinePLotVals = np.empty(shape=(4,4,4,200))

    for i in range(3):
        for j in range(2):
            df_kde=df_active.copy(deep=True)
            df_kde=df_kde[df_kde['CIR']==KDP_nu_name[j]]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            df_kde.dropna(subset=['Date Of Peak'],inplace=True)
            
            
                
                
            
            for k in range(3):
                df_kdep = df_kde.copy(deep=True)
                
                df_kdep = df_kdep[df_kdep['KDPLambda']=="SD_RARF" ]
                
                df_kdep = df_kdep[df_kdep['Vaccine Rate']==KDP_c_p_name[k]]   
                
                
                
                
                ## Add Jitters to the Current Dataframe
                stdev_DOP = JitterDate * (max(df_kde['Date Of Peak']) - min(df_kde['Date Of Peak']))
                df_kde['Date Of Peak'] =  df_kde['Date Of Peak'] + np.random.randn(len(df_kde['Date Of Peak'])) * stdev_DOP

                stdev_Case = JitterNum * (max(df_kde['Number At Peak']) - min(df_kde['Number At Peak']))
                df_kde['Number At Peak'] =  df_kde['Number At Peak'] + np.random.randn(len(df_kde['Number At Peak'])) * stdev_Case


                scat = sns.scatterplot(ax=ax[i,j],x='Date Of Peak',y='Number At Peak',data=df_kdep,y_jitter=0.2,color=colorVal[k],label=Vacc[k],marker=marker[k],s=80,palette=colorVal,markers=True )
    #             for line in range(0,df_kdep.shape[0]):
    #                  plt.text(df_kdep['Date Of Peak'][line]+0.2, df_kdep['Number At Peak'][line],df_kdep['Scenario'][line],size='large')

                plt.yscale(value="log")
            ax[i,j].set_xticks(xt);
            ax[i,j].set_xticklabels(xtl);
            ax[i,j].set_yticks(xt_C);
            ax[i,j].set_yticklabels(xtl_C);
            ax[i,j].tick_params(axis='x', rotation=60)
            ax[i,j].text(.48,.9,KDPImm[i] +", "+ KDPCIR[j],horizontalalignment='center',transform=ax[i,j].transAxes)
            ax[i,j].legend(title='Vaccination Rate',fancybox=True,ncol=2)
            handles, labels = ax[i,j].get_legend_handles_labels()
            ax[i,j].set_ylim([1e-2,1e9])
            ax[i,j].get_legend().remove()
    #             ax.legend(..., ncol=2)
            ax[i,j].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,j].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)

    ## Add the y labels in 1st Column
    ax[0,0].set_ylabel("Active cases at peak")
    ax[1,0].set_ylabel("Active cases at peak")
    ax[2,0].set_ylabel("Active cases at peak")

    ## Add the x axis labels in last row
    ax[2,0].set_xlabel("Date of peak")
    ax[2,1].set_xlabel("Date of peak")

    plt.legend(handles, labels, loc='upper center',title='',fancybox=True,ncol=3,bbox_to_anchor=(-0.1,3.45))
    plt.subplots_adjust(wspace=0.04, hspace=0.085)

    name = "Scatter"
    name = name + "_" + plotName 
    plt.savefig(name + ".png",dpi=300,bbox_inches='tight')

    plt.show()



In [ ]:
###--- PLOTTER ----     ####
### @@ GROUP SCATTER PLOTTER    ###



###################### _____________________GROUP SCATTER PLOTTERS ______________________________ ##############################
        
import warnings
warnings.filterwarnings('ignore')


def SCATTER_plotter_Group(df_active,plotName):
    
    from scipy.stats import norm
    import numpy
    import sklearn
    from sklearn.neighbors import KernelDensity
    from sklearn.utils.fixes import parse_version
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import LeaveOneOut
    from sklearn import preprocessing
    import matplotlib.patches as mpatches
    fig, ax = plt.subplots(3,4,figsize=(11,8),sharex=True,sharey=True)

    ####### -- PLotter Parameters -- ######
    plt.rcParams["text.usetex"] = False
    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.serif"] = ["Computer Modern"]
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    plt.rcParams["font.serif"]
    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    xtl = []
    xt = [396,427,457,488,518,549,580,608,639,670]   ## 1st days of the months hardcoded 
    xtl = ['   Aug\'21','','   Oct\'21','','Dec\'21', '','Feb\'22','','Apr\'22','No wave-3']



    ## Set plotting parameters   -- Number at Peak
    markersizep = 1.9
    nTicks_C = 10
    startDay_C = 0
    endDay_C = 4e6
    xt_C = [1e-2,1e0,1e2,1e4,1e6,1e8,1e9]
    xtl_C = ["0",r"$10^{0}$",r"$10^{2}$",r"$10^{4}$",r"$10^{6}$",r"$10^{8}$",r"$10^{9}$"];


    #### PLotting Colors and other Parameters 
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    kernel = 'epanechnikov'
    KDPCIR=["CIR 40", "CIR 50"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDP_lambda_Array = ["SD-Normal","SD-Bad","SD-Worse","SD-Overall"]
    BackGroundColor = ["#ffffff","#ffffff","#ffffff","#ffffff"]
    JitterDate = 0.01
    JitterNum = 0.00000007
        #LinePLotVals = np.empty(shape=(4,4,4,200))
    #LinePLotVals = np.empty(shape=(4,4,4,200))

    for i in range(3):      #Immune Escape
        for j in range(4):  #  SD Factor
            df_kde=df_active.copy(deep=True)
                        ## Add Jitters to the Current Dataframe
            stdev_DOP = JitterDate * (max(df_kde['Date Of Peak']) - min(df_kde['Date Of Peak']))
            df_kde['Date Of Peak'] =  df_kde['Date Of Peak'] + np.random.randn(len(df_kde['Date Of Peak'])) * stdev_DOP

            stdev_Case = JitterNum * (max(df_kde['Number At Peak']) - min(df_kde['Number At Peak']))
            df_kde['Number At Peak'] =  df_kde['Number At Peak'] + np.random.randn(len(df_kde['Number At Peak'])) * stdev_Case
            
            df_kde=df_kde[df_kde['CIR']=="CIR40"]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            
            if(j !=3):
                df_kde = df_kde[df_kde['KDPLambda']== KDP_lambda_name[j]]
            
            df_kde.dropna(subset=['Date Of Peak'],inplace=True)
            
            
                
                
            
            for k in range(3):
                df_kdep = df_kde.copy(deep=True)
                
                
                
                df_kdep = df_kdep[df_kdep['Vaccine Rate']==KDP_c_p_name[k]]   
                
                


                scat = sns.scatterplot(ax=ax[i,j],x='Date Of Peak',y='Number At Peak',data=df_kdep,y_jitter=0.2,color=colorVal[k],label=Vacc[k],marker=marker[k],s=80,palette=colorVal,markers=True,edgecolor = "#333333" )
    #             for line in range(0,df_kdep.shape[0]):
    #                  plt.text(df_kdep['Date Of Peak'][line]+0.2, df_kdep['Number At Peak'][line],df_kdep['Scenario'][line],size='large')

                plt.yscale(value="log")
            ax[i,j].set_xticks(xt);
            ax[i,j].set_xticklabels(xtl);
            ax[i,j].set_yticks(xt_C);
            ax[i,j].set_yticklabels(xtl_C);
            ax[i,j].tick_params(axis='x', rotation=60)
            ax[i,j].text(.48,.9,KDPImm[i] +", "+ KDP_lambda_Array[j],horizontalalignment='center',transform=ax[i,j].transAxes)
            ax[i,j].legend(title='Vaccination Rate',fancybox=True,ncol=2)
            handles, labels = ax[i,j].get_legend_handles_labels()
            ax[i,j].set_ylim([1e-2,1e9])
            ax[i,j].get_legend().remove()
    #             ax.legend(..., ncol=2)
            ax[i,j].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,j].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
            ## ABckground Color
            ax[i,j].set_facecolor(BackGroundColor[j]);

    ## Add the y labels in 1st Column
    ax[0,0].set_ylabel("Active cases at peak")
    ax[1,0].set_ylabel("Active cases at peak")
    ax[2,0].set_ylabel("Active cases at peak")

    ## Add the x axis labels in last row
    ax[2,0].set_xlabel("Date of peak")
    ax[2,1].set_xlabel("Date of peak")
    ax[2,2].set_xlabel("Date of peak")
    ax[2,3].set_xlabel("Date of peak")

    plt.legend(handles, labels, loc='upper center',title='',fancybox=True,ncol=3,bbox_to_anchor=(-1.4,3.45))
    plt.subplots_adjust(wspace=0.04, hspace=0.085)

    name = "Scatter"
    name = name + "_" + plotName 
    plt.savefig(name + ".png",dpi=300,bbox_inches='tight')

    plt.show()

SCATTER_plotter_Group(df_active,"TotalActive_Group")

In [ ]:
###--- PLOTTER ----     ####
### @@ AgeWise GROUP SCATTER PLOTTER    ###

###################### _____________________GROUP SCATTER PLOTTERS [[[ AGE WISE ]]]______________________________ ##############################
        
import warnings
warnings.filterwarnings('ignore')


def SCATTER_plotter_Group_AgeWise(df_active,plotName):
    
    from scipy.stats import norm
    import numpy
    import sklearn
    from sklearn.neighbors import KernelDensity
    from sklearn.utils.fixes import parse_version
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import LeaveOneOut
    from sklearn import preprocessing
    import matplotlib.patches as mpatches
    fig, ax = plt.subplots(1,1,figsize=(11,8),sharex=True,sharey=True)

    ####### -- PLotter Parameters -- ######
    plt.rcParams["text.usetex"] = False
    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.serif"] = ["Computer Modern"]
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    plt.rcParams["font.serif"]
    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    xtl = []
    xt = [396,427,457,488,518,549,580,608,639,670]   ## 1st days of the months hardcoded 
    xtl = ['   Aug\'21','','   Oct\'21','','Dec\'21', '','Feb\'22','','Apr\'22','No wave-3']



    ## Set plotting parameters   -- Number at Peak
    markersizep = 1.9
    nTicks_C = 10
    startDay_C = 0
    endDay_C = 4e6
    xt_C = [1e-2,1e0,1e2,1e4,1e6,1e8,1e9]
    xtl_C = ["0",r"$10^{0}$",r"$10^{2}$",r"$10^{4}$",r"$10^{6}$",r"$10^{8}$",r"$10^{9}$"];


    #### PLotting Colors and other Parameters 
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    kernel = 'epanechnikov'
    KDPCIR=["CIR 40", "CIR 50"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDP_lambda_Array = ["SD-Normal","SD-Bad","SD-Worse","SD-Overall"]
    JitterDate = 0.01
    JitterNum = 0.00000007
        #LinePLotVals = np.empty(shape=(4,4,4,200))
    #LinePLotVals = np.empty(shape=(4,4,4,200))

    for i in range(3):      #Immune Escape
        for j in range(4):  #  SD Factor
            df_kde=df_active.copy(deep=True)
                        ## Add Jitters to the Current Dataframe
            stdev_DOP = JitterDate * (max(df_kde['Date Of Peak']) - min(df_kde['Date Of Peak']))
            df_kde['Date Of Peak'] =  df_kde['Date Of Peak'] + np.random.randn(len(df_kde['Date Of Peak'])) * stdev_DOP

            stdev_Case = JitterNum * (max(df_kde['Number At Peak']) - min(df_kde['Number At Peak']))
            df_kde['Number At Peak'] =  df_kde['Number At Peak'] + np.random.randn(len(df_kde['Number At Peak'])) * stdev_Case
            
            df_kde=df_kde[df_kde['CIR']=="CIR40"]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            
            if(j !=3):
                df_kde = df_kde[df_kde['KDPLambda']== KDP_lambda_name[j]]
            
            df_kde.dropna(subset=['Date Of Peak'],inplace=True)
            
            
                
                
            
            for k in range(3):
                df_kdep = df_kde.copy(deep=True)
                
                
                
                df_kdep = df_kdep[df_kdep['Vaccine Rate']==KDP_c_p_name[k]]   
                
                


                scat = sns.scatterplot(ax=ax[i,j],x='Date Of Peak',y='Number At Peak',data=df_kdep,y_jitter=0.2,color=colorVal[k],label=Vacc[k],marker=marker[k],s=80,palette=colorVal,markers=True,edgecolor = "#333333" )
    #             for line in range(0,df_kdep.shape[0]):
    #                  plt.text(df_kdep['Date Of Peak'][line]+0.2, df_kdep['Number At Peak'][line],df_kdep['Scenario'][line],size='large')

                plt.yscale(value="log")
            ax[i,j].set_xticks(xt);
            ax[i,j].set_xticklabels(xtl);
            ax[i,j].set_yticks(xt_C);
            ax[i,j].set_yticklabels(xtl_C);
            ax[i,j].tick_params(axis='x', rotation=60)
            ax[i,j].text(.48,.9,KDPImm[i] +", "+ KDP_lambda_Array[j],horizontalalignment='center',transform=ax[i,j].transAxes)
            ax[i,j].legend(title='Vaccination Rate',fancybox=True,ncol=2)
            handles, labels = ax[i,j].get_legend_handles_labels()
            ax[i,j].set_ylim([1e-2,1e9])
            ax[i,j].get_legend().remove()
    #             ax.legend(..., ncol=2)
            ax[i,j].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            ax[i,j].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
            
            ## ABckground Color
            ax[i,j].set_facecolor(BackGroundColor[j]);

    ## Add the y labels in 1st Column
    ax[0,0].set_ylabel("Active cases at peak")
    ax[1,0].set_ylabel("Active cases at peak")
    ax[2,0].set_ylabel("Active cases at peak")

    ## Add the x axis labels in last row
    ax[2,0].set_xlabel("Date of peak")
    ax[2,1].set_xlabel("Date of peak")
    ax[2,2].set_xlabel("Date of peak")
    ax[2,3].set_xlabel("Date of peak")

    plt.legend(handles, labels, loc='upper center',title='',fancybox=True,ncol=3,bbox_to_anchor=(-1.4,3.45))
    plt.subplots_adjust(wspace=0.04, hspace=0.085)

    name = "Scatter"
    name = name + "_" + plotName 
    plt.savefig(name + ".png",dpi=300,bbox_inches='tight')

    plt.show()



In [ ]:
###--- PLOTTER ----     ####
### @@ AgeWise  SCATTER PLOTTER    ###
# Normal Agewise Scatter Plot
###################### _____________________GROUP SCATTER PLOTTERS [[[ AGE WISE ]]]______________________________ ##############################
        
import warnings
warnings.filterwarnings('ignore')
import numpy
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def SCATTER_plotter_AgeWise(df_active,x_axis,y_axis,plotName,NeedsJitter):
        
    fig, ax = plt.subplots(figsize=(11,8))
    

    plt.rcParams["font.serif"]
    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


    xt = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75]
    xtl = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75]

    #### PLotting Colors and other Parameters 
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    kernel = 'epanechnikov'
    KDPCIR=["CIR 40", "CIR 50"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDP_lambda_Array = ["SD-Normal","SD-Bad","SD-Worse","SD-Overall"]
    JitterDate = 0.01
    JitterNum = 0.00000007
    

    ## Copy the DataFrame 
    df_kde=df_active.copy(deep=True)
    
    ## Add Jitters to the Current Dataframe
    if(NeedsJitter):
        stdev_DOP = JitterDate * (max(df_kde['Date Of Peak']) - min(df_kde['Date Of Peak']))
        df_kde['Date Of Peak'] =  df_kde['Date Of Peak'] + np.random.randn(len(df_kde['Date Of Peak'])) * stdev_DOP

        stdev_Case = JitterNum * (max(df_kde['Number At Peak']) - min(df_kde['Number At Peak']))
        df_kde['Number At Peak'] =  df_kde['Number At Peak'] + np.random.randn(len(df_kde['Number At Peak'])) * stdev_Case

    ## Filter out the No Peak Cases
    df_kde = df_kde[df_kde["Date Of Peak"] != NOWAVEPEAKDATE]
    

    for k in range(3):
        df_kdep = df_kde[df_kde['Vaccine Rate']==KDP_c_p_name[k]]   


        scat = sns.scatterplot(x=x_axis,y=y_axis,data=df_kdep,y_jitter=0.2,color=colorVal[k],label=Vacc[k],marker=marker[k],s=80,palette=colorVal,markers=True,edgecolor = "#333333" )
#             for line in range(0,df_kdep.shape[0]):
#                  plt.text(df_kdep['Date Of Peak'][line]+0.2, df_kdep['Number At Peak'][line],df_kdep['Scenario'][line],size='large')

        
    plt.xticks(xt,xtl);
    plt.yticks(xt,xtl);
    ax.tick_params(axis='x', rotation=60)
    ax.text(.48,.9,"Test",horizontalalignment='center',transform=ax.transAxes)
    plt.legend(title='Vaccination Rate',fancybox=True,ncol=2)
    handles, labels = ax.get_legend_handles_labels()
    ax.get_legend().remove()
#             ax.legend(..., ncol=2)
    ax.grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
    ax.grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)


    ## Add the y labels in 1st Column
    ax.set_ylabel("18 Fraction at Peak")

    ## Add the x axis labels in last row
    ax.set_xlabel("11 Fraction at Peak")


#     plt.legend(handles, labels, loc='upper center',title='',fancybox=True,ncol=3,bbox_to_anchor=(-1.4,3.45))

    name = "Scatter"
    name = name + "_" + plotName 
    plt.savefig(name + ".png",dpi=300,bbox_inches='tight')

    plt.show()

SCATTER_plotter_AgeWise(df_nucleation_total,"11FracAtPeak","18FracAtPeak","scatter_age11v18",False)

In [ ]:
###--- PLOTTING CELL ----     ####
### @@ TIMESERIES PLOT    ###



####### Create a TIMESERIES PLOT ###########################
import matplotlib.pyplot as plt
## get Distribution of Data for each of the Rows ( Prefereble in Log Scale )
###

def Histogram(array,bins,start,end,result):
    
    lenX = len(array);
    
    binWidth = (end -  start)/bins;

    for i in range(lenX):
        val    = array[i];
        binNo  = int(val/binWidth);
        
        result[binNo]+=1;
    
    return result;

def TimeSeriesPlotter(DataFrame,plotname,PlottingParameters, LogParam):

    N_days_Experimentation  = ROWS
    
    ## Obtain the parameters from Disct
    Bandwidth = PlottingParameters["bandwidth"]
    N_bins    = PlottingParameters["N_bins"]
    start     = PlottingParameters["Start"]
    end       = PlottingParameters["End"]

    TimeSeriesData = np.zeros([N_bins,ROWS+1],dtype=np.float)
    
    ## Compute the distribution for each of the data

    for i in range(ROWS+1):
        #Get the Data of that Col
        data = DataFrame[[i]].copy(deep=True)
        data = data+1;   ##### To remove the log errors whle trying to operate on zero
        data = data.to_numpy().flatten()

        if(LogParam == True):
            data = np.log10(data)
        
        # Plot a SNS Histogram and obtain the kdeline
        
        Histogram(data,N_bins,start,end,TimeSeriesData[:,i]);
        
#         print(TimeSeriesData[:,i])
        
#         if(sum(TimeSeriesData[:,i]) != 972):
#             print(" Some Thing Wrong")
        
    
    ## Normalise t
    MaxValue       = TimeSeriesData.max()
    MinValue       = TimeSeriesData.min()
    TimeSeriesData = TimeSeriesData/MaxValue
    
    MaxValueNormalised = TimeSeriesData.max()
    print("MAx Val, norm : ", MaxValueNormalised)
    ## Plot the Data
    X = np.linspace(0,ROWS,ROWS+1)
    Y = np.linspace(start,end,N_bins)
    levels = np.linspace(0,0.08,60)
    
    origin = 'lower'
    
    yt = [0,1,2,3,4,5,6,7]
    ytl = xtl_C = ["0",r"         $10^{1}$",r"         $10^{2}$",r"    $10^{3}$",r"    $10^{4}$",r"    $10^{5}$",r"    $10^{6}$",r"    $10^{7}$"];
    
    xt = [0,92,182,274,364,457,549,639,700]
    xtl = ["Jul'20","Oct'20", "Jan'21","Apr'21","Jun'21","Oct'21","Jan'22","Mar'22","Jun'22"]
    
    plt.figure(figsize=(11,8))
    ax = plt.contourf(X,Y,TimeSeriesData,levels,
                  #[-1, -0.1, 0, 0.1],
                  #alpha=0.5,
                      origin='upper',cmap='jet'
                  )
    ax.cmap.set_under('red')
    ax.cmap.set_over('cyan')
    
    plt.ylabel("Total Active Cases")
    plt.yticks(yt,ytl)
    plt.colorbar(ax)
    plt.xticks(xt,xtl)
    plt.tick_params(axis='x', rotation=60);
    
    plt.tight_layout()
    
##Plotting parameters
        
PlottingParameters = {
    'bandwidth': 0.3,
    'N_bins': 200,
    'Start' : 0,
    'End'   : 8
}       
        
TimeSeriesPlotter(df_active,"timeSeries_DailyActive",PlottingParameters,True)

In [ ]:
### PLOTTER SCRIPT #####
## Pair Plot 5-Tuple ###
## 5*5 Pair #

def sns_PairPlot(DataFrame,VarArray,plotTitle,CustomXTicks):
    


    plt.rcParams["font.serif"]
    SMALL_SIZE = 12
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


    xt = [0,0.1,0.2,0.3,0.4,0.5,0.6]
    xtl = [0,0.1,0.2,0.3,0.4,0.5,0.6]

    #### PLotting Colors and other Parameters 
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    marker  = ['o','h','^']
    kernel = 'epanechnikov'
    KDPCIR=["CIR 40", "CIR 50"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
#     XLabel = ["Ratio 0-11 Year","Ratio 11-18 Year","Ratio 18-45 Year","Ratio 45-60 Year","Ratio 60+"]
#     YLabel = ["Ratio 0-11 Year","Ratio 11-18 Year","Ratio 18-45 Year","Ratio 45-60 Year","Ratio 60+"]
    XLabel = ["Ratio 18Plus" , "Ratio 18Minus "]
    YLabel = ["Ratio 18Plus" , "Ratio 18Minus "]
    KDP_lambda_Array = ["SD-Normal","SD-Bad","SD-Worse","SD-Overall"]
    
    DataFrame = DataFrame[DataFrame["Date Of Peak"] != NOWAVEPEAKDATE].copy(deep=True)
    
    DataFrame["VacineAndSD"] = DataFrame["Vaccine Rate"] + "_" + DataFrame["SD"]
    
    print("Before Plot")
#     g=sns.pairplot(data=DataFrame,vars=VarArray, hue='SD',markers=marker,palette=colorVal,plot_kws={"s": 60})
    g=sns.pairplot(data=DataFrame,vars=VarArray, hue="VacineAndSD")
    
    handles = g._legend_data.values()
    labels = g._legend_data.keys()
#     g._legend.remove()
    
    ### X labels
    for j in range(len(VarArray)):
        g.axes[len(VarArray)-1][j].set_xlabel(XLabel[j])
        g.axes[len(VarArray)-1][j].tick_params(axis='x', rotation=60);
        
        
    
    ### Y labels
    for i in range(len(VarArray)):
        g.axes[i][0].set_ylabel(XLabel[i])
    
    for i in range(len(VarArray)):
        for j in range(len(VarArray)):
            if(CustomXTicks):
                g.axes[i][j].set_xticks(xt);
            g.axes[i,j].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
            g.axes[i,j].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
    
            
        
    
    plt.grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
    plt.grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
    
    
#     plt.legend(handles, labels, loc='upper center',title='',fancybox=True,ncol=3,bbox_to_anchor=(-1.5,5.65)

#     plt.legend(handles, labels, loc='upper center',title='',ncol=3,bbox_to_anchor=(-.5,-1.98))
               
#     plt.tight_layout()
    
    name = "PairPlot_" + plotTitle + ".png"
    plt.savefig(name,dpi=400,bbox_inches='tight')
    
    print("After Plot")
    

    plt.show()
    pass;
## 
# Array = ['11FracAtPeak','18FracAtPeak','45FracAtPeak','60FracAtPeak','60plusFracAtPeak']

# sns_PairPlot(df_nucleation_total,Array,"allAgeGroup_withoutXticks",False)
# sns_PairPlot(df_nucleation_total,Array,"allAgeGroup",True)

Array = ['18plusFracAtPeak','18minusFracAtPeak']
sns_PairPlot(df_nucleation_total,Array,"AgeGroup_18",False)


In [ ]:
###### PLOTTER SCRIPT ################
### REL PLot   #####


### Reshape DataFrame
def Relplotter(DataFrame, SD_Col_Val, plotExtension,ciBool):
    df_melt = df_active.copy(deep=True)
    listCol = list(df_active.columns.values)[731:]
    df_melt = df_melt.melt(id_vars=listCol, \
                var_name="Day", \
                value_name="Total")

    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['axes.labelsize']=16
    

    plt.rcParams["font.serif"]
    SMALL_SIZE = 16
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 16

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    ### PLotting Parameters
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    ## Filter the Data
    df_melt=df_melt[df_melt['Day']>300]
    df_melt=df_melt[df_melt['SD']==SD_Col_Val]


    if(ciBool):
    ### Plotting Script
        relPlot = sns.relplot(data=df_melt, kind="line",
                            x="Day", y="Total",
                            hue="Vaccine Rate",col="KDPwsat2",palette=colorVal,facet_kws=dict(legend_out=False))
    else:
        relPlot = sns.relplot(data=df_melt, kind="line",ci=None,
                            x="Day", y="Total",
                            hue="Vaccine Rate",col="KDPwsat2",palette=colorVal,facet_kws=dict(legend_out=False))

#     handles = relPlot._legend.
#     labels = relPlot._legend_data.keys()
#     print(labels)
#     print(max(df_melt['Total']))
    h,l = relPlot.axes[0][0].get_legend_handles_labels()
    relPlot.axes[0][0].legend_.remove()


    ## yTicks
    yt = [0,1,2,3,4,5,6,7,8]

    ## Xticks
#     xt = [304,335,365,396,427,457,488,518,549,580,608,639,670,700]   ## 1st days of the months hardcoded 
#     xtl = ['   May\'21','   Jun\'21','   Jul\'21','   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','May\'22','Jun\'22']

    xt = [304,365,427,488,549,608,670]   ## 1st days of the months hardcoded 
    xtl = ['   May\'21','   Jul\'21','   Sep\'21','Nov\'21', 'Jan\'22','Mar\'22','May\'22']

    SDArr = {"SD_RA" : 0,
            "SD_RAR" : 0.1,
            "SD_RARF": 0.2}

    relPlot.axes[0][0].set_ylabel("Daily confirmed cases")
    k = relPlot.axes[0][0].get_yticks()
    
    def YTicks(array):
        array = [item for item in array if item >= 0]
        Ylabel = []
        for i in range(len(array)):
            if(array[i]/1000000 >=1 ): # Millions
                Ylabel.append(str(array[i]/1000000) + "M")

            elif(array[i] !=0):                       # Thousands
                Ylabel.append(str((int(array[i]/1000))) + "K")
            else:
                Ylabel.append("0")
        return array,Ylabel
    yt,ytl = YTicks(k)
    

    for i in range(3):
        relPlot.axes[0][i].set_xlabel("Time")                   ## Xlabel
        relPlot.axes[0][i].set_xticks(xt)                       ## XTicks 
        relPlot.axes[0][i].set_xticklabels(xtl)                 ## XTick Labels
        relPlot.axes[0][i].tick_params(axis='x', rotation=60);  ## XTicks Rotation
        relPlot.axes[0][i].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
        relPlot.axes[0][i].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
        relPlot.axes[0][i].set_yticks(yt)
        relPlot.axes[0][i].text(.48,.92,KDPImm[i],horizontalalignment='center',transform=relPlot.axes[0][i].transAxes)
        relPlot.axes[0][i].set_title("")
        relPlot.axes[0][i].spines['right'].set_visible(True)
        relPlot.axes[0][i].spines['top'].set_visible(True)


    relPlot.axes[0][0].set_yticks(yt)                       ## YTicks 
    relPlot.axes[0][0].set_yticklabels(ytl)                 ## YTick Labels
    
    l = Vacc
    relPlot.fig.legend(h, l, loc='upper center',title='',ncol=3,bbox_to_anchor=(0.5,1.05), frameon=False)
    
    name = "Relplot_" + plotExtension + "1.png"
    plt.tight_layout()
    plt.savefig(name,dpi=300,bbox_inches='tight');
    
Relplotter(df_antibody,"SD_RA","SD_RA",True)
# Relplotter(df_active,"SD_RAR","SD_RAR",True)
# Relplotter(df_active,"SD_RARF","SD_RARF",True)

In [ ]:
###### PLOTTER SCRIPT--Inset Divij  ################
### REL PLot   #####
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

### Reshape DataFrame
def Relplotter(DataFrame, SD_Col_Val, plotExtension,ciBool):
    
    df_melt = df_active.copy(deep=True)
    listCol = list(df_active.columns.values)[731:]
    df_melt = df_melt.melt(id_vars=listCol, \
                var_name="Day", \
                value_name="Total")

    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['axes.labelsize']=16
    

    plt.rcParams["font.serif"]
    SMALL_SIZE = 16
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 16

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    ### PLotting Parameters
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    KDPImm = ["Jul'21", "Sep'21","Nov'21" ]
    ## Filter the Data
    df_melt=df_melt[df_melt['Day']>300]
    df_melt=df_melt[df_melt['SD']==SD_Col_Val]
    
    df_melt_copy=df_melt.copy(deep=True)
    df_melt_copy=df_melt_copy[df_melt_copy['KDPwsat2']=='ImmunEscp_Jul']
    

    if(ciBool):
    ### Plotting Script
        relPlot = sns.relplot(data=df_melt_copy,
                            x="Day", y="Total",
                            hue="Vaccine Rate",kind='line',palette=colorVal)#,facet_kws=dict(legend_out=False))
    else:
#         fig, ax = plt.subplots()
       relPlot= sns.relplot(data=df_melt_copy,ci=None,
                            x="Day", y="Total",
                            hue="Vaccine Rate",kind='line',palette=colorVal,facet_kws=dict(legend_out=False))
       
#         axins = inset_axes(ax,  "30%", "40%" ,loc="lower right", borderpad=3)
#         sns.relplot(data=df_melt_copy, kind="line",ci=None,
#                             x="Day", y="Total",
#                             hue="Vaccine Rate",palette=colorVal,facet_kws=dict(legend_out=False),ax=axins)

#     handles = relPlot._legend.
#     labels = relPlot._legend_data.keys()
#     print(labels)
#     print(max(df_melt['Total']))
#     h,l = relPlot.axes[0][0].get_legend_handles_labels()
#     relPlot.axes[0][0].legend_.remove()


#     ## yTicks
#     yt = [0,1,2,3,4,5,6,7,8]

#     ## Xticks
# #     xt = [304,335,365,396,427,457,488,518,549,580,608,639,670,700]   ## 1st days of the months hardcoded 
# #     xtl = ['   May\'21','   Jun\'21','   Jul\'21','   Aug\'21','   Sep\'21','   Oct\'21','Nov\'21','Dec\'21', 'Jan\'22','Feb\'22','Mar\'22','Apr\'22','May\'22','Jun\'22']

#     xt = [304,365,427,488,549,608,670]   ## 1st days of the months hardcoded 
#     xtl = ['   May\'21','   Jul\'21','   Sep\'21','Nov\'21', 'Jan\'22','Mar\'22','May\'22']

#     SDArr = {"SD_RA" : 0,
#             "SD_RAR" : 0.1,
#             "SD_RARF": 0.2}

#     relPlot.axes[0][0].set_ylabel("Daily confirmed cases")
#     k = relPlot.axes[0][0].get_yticks()
    
#     def YTicks(array):
#         array = [item for item in array if item >= 0]
#         Ylabel = []
#         for i in range(len(array)):
#             if(array[i]/1000000 >=1 ): # Millions
#                 Ylabel.append(str(array[i]/1000000) + "M")

#             elif(array[i] !=0):                       # Thousands
#                 Ylabel.append(str((int(array[i]/1000))) + "K")
#             else:
#                 Ylabel.append("0")
#         return array,Ylabel
#     yt,ytl = YTicks(k)
    

#     for i in range(1):
#         relPlot.axes[0][i].set_xlabel("Time")                   ## Xlabel
#         relPlot.axes[0][i].set_xticks(xt)                       ## XTicks 
#         relPlot.axes[0][i].set_xticklabels(xtl)                 ## XTick Labels
#         relPlot.axes[0][i].tick_params(axis='x', rotation=60);  ## XTicks Rotation
#         relPlot.axes[0][i].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
#         relPlot.axes[0][i].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
#         relPlot.axes[0][i].set_yticks(yt)
#         relPlot.axes[0][i].text(.48,.92,KDPImm[i],horizontalalignment='center',transform=relPlot.axes[0][i].transAxes)
#         relPlot.axes[0][i].set_title("")
#         relPlot.axes[0][i].spines['right'].set_visible(True)
#         relPlot.axes[0][i].spines['top'].set_visible(True)


#     relPlot.axes[0][0].set_yticks(yt)                       ## YTicks 
#     relPlot.axes[0][0].set_yticklabels(ytl)                 ## YTick Labels
    
#     l = Vacc
#     relPlot.fig.legend(h, l, loc='upper center',title='',ncol=3,bbox_to_anchor=(0.5,1.05), frameon=False)
    
#     name = "Relplot_" + plotExtension + "1.png"
#     plt.tight_layout()
#     plt.savefig(name,dpi=300,bbox_inches='tight');
    return relPlot
    
rp = Relplotter(df_antibody,"SD_RA","SD_RA",True)
#rp = Relplotter(df_active,"SD_RAR","SD_RAR",True)
# Relplotter(df_active,"SD_RARF","SD_RARF",True)

In [ ]:
```

In [ ]:
rp.data

In [ ]:
from scipy import stats
meanrp=[]
ciurp=[]
cilrp=[]

for i in range(301,731):
    rp1=rp.data.copy(deep=True)
    rpp=rp1[rp1['hue']=='Vaccine_PresentRate'].copy(deep=True)
    rpp=rpp[rpp['x']==i]
    meanrp.append(np.mean(rpp['y']))
    df=rpp['y'].shape[0] - 1
    
    ci=stats.t.interval(alpha=0.95,df=df,loc=np.mean(rpp['y']),scale=scipy.stats.sem(rpp['y']))
    cilrp.append(ci[0])
    ciurp.append(ci[1])
    
cilrp[3:5]

In [ ]:
x=np.arange(301,731,1)
fig, ax = plt.subplots()
ax.plot(x,meanrp,color='red')
ax.fill_between(x,cilrp,ciurp,color='blue',alpha=0.1)
x1=np.arange(500,731,1)
axins = inset_axes(ax,  "30%", "40%" ,loc="upper right", borderpad=3)
axins.plot(x1,meanrp[199:430],color='red')
axins.fill_between(x1,cilrp[199:430],ciurp[199:430],color='blue',alpha=0.1)

In [ ]:
###### COMPUTE #####################
## AgeWise Data for RELPLOT ##

df_nucleation_18minus = df_nucleation_18.iloc[:,:736].copy(deep=True)
df_nucleation_18minus.iloc[:,:731] += df_nucleation_11.iloc[:,:731]
df_nucleation_18plus = df_nucleation_45.iloc[:,:736].copy(deep=True)
df_nucleation_18plus.iloc[:,:731] +=  df_nucleation_60.iloc[:,:731] 

df_melt_18plus = df_nucleation_18plus.copy(deep=True)
listCol = list(df_nucleation_18plus.columns.values)[731:]
df_melt_18plus = df_melt_18plus.melt(id_vars=listCol, \
            var_name="Day", \
            value_name="18Plus")
df_melt1_18minus = df_nucleation_18minus.copy(deep=True)
listCol = list(df_nucleation_18minus.columns.values)[731:]
df_melt_18minus = df_melt1_18minus.melt(id_vars=listCol, \
            var_name="Day", \
            value_name="18Minus")
df_melt_60plus = df_nucleation_60plus.copy(deep=True)
listCol = list(df_melt_60plus.columns.values)[731:]
df_melt_60plus = df_melt_60plus.melt(id_vars=listCol, \
            var_name="Day", \
            value_name="60Plus")

# Copy all contents to single dataframe
df_nucleation_melt_Age = df_melt_18minus.copy(deep=True)
df_nucleation_melt_Age["18Plus"] = df_melt_18plus["18Plus"].to_list()
df_nucleation_melt_Age["60Plus"] = df_melt_60plus["60Plus"].to_list()
#dump = df_melt_Age_18.copy(deep=True)

## Melt the final Dataframe
listCol = list(df_nucleation_melt_Age.columns.values)[:6]
df_nucleation_melt_Age = df_nucleation_melt_Age.melt(id_vars=listCol, \
            var_name="AgeGroup", \
            value_name="AgeTotal")



In [ ]:
#### PLOTTER SCRIPT ###########
### Agewise Relplot #########
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def agewise_Relplot(dataframe,plotExtension,VAC_RATE,ciFlag):
    dat = dataframe.copy(deep=True)
    dat = dat[dat["Day"] > 300]
    dat = dat[dat["Vaccine Rate"] == VAC_RATE]

    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['axes.labelsize']=16

    plt.rcParams["font.serif"]
    SMALL_SIZE = 16
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 16

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    Vacc=["Present Rate","1.5x Present Rate","2x Present Rate"]
    Age=["Age : 0-17","Age: 18-60","Age : 60+"]
    Sd=["SD-Normal","SD-Bad","SD-Worse"]
    KDPImm=["IENV-Jul'21","IENV-Sep'21","IENV-Nov'21"]
    
    if(not ciFlag):
        relPlot = sns.relplot(data=dat, kind="line",ci=None, \
                                x="Day", y="AgeTotal", \
                                hue="AgeGroup",col="KDPwsat2",label=Vacc,palette=colorVal,facet_kws=dict(legend_out=False))
    else:
        relPlot = sns.relplot(data=dat, kind="line", \
                                x="Day", y="AgeTotal", \
                                hue="AgeGroup",col="KDPwsat2",label=Vacc,palette=colorVal,facet_kws=dict(legend_out=False))
        axins = inset_axes(relPlot.axes[0][0],  "30%", "40%" ,loc="upper right", borderpad=3)
        sns.relplot(x='Day',y='AgeTotal', 
                   data=dat, hue="AgeGroup",ax=axins)#x_estimator=np.mean,
    
    h,l = relPlot.axes[0][0].get_legend_handles_labels()
    relPlot.axes[0][0].legend_.remove()
    
    yt = [0,1,2,3,4,5,6,7,8]

    ## Xticks
    xt = [304,365,427,488,549,608,670]   ## 1st days of the months hardcoded 
    xtl = ['   May\'21','   Jul\'21','   Sep\'21','Nov\'21', 'Jan\'22','Mar\'22','May\'22']
    
    SDArr = {"SD_RA" : 0,
            "SD_RAR" : 0.1,
            "SD_RARF": 0.2}

    relPlot.axes[0][0].set_ylabel("Daily confirmed cases")
    k = relPlot.axes[0][0].get_yticks()

    def YTicks(array):
        array = [item for item in array if item >= 0]
        Ylabel = []
        for i in range(len(array)):
            if(array[i]/1000000 >=1 ): # Millions
                Ylabel.append(str(array[i]/1000000) + "M")

            elif(array[i] !=0):                       # Thousands
                Ylabel.append(str((int(array[i]/1000))) + "K")
            else:
                Ylabel.append("0")
        return array,Ylabel
    yt,ytl = YTicks(k)


    for i in range(3):
        relPlot.axes[0][i].set_xlabel("Time")                   ## Xlabel
        relPlot.axes[0][i].set_xticks(xt)                       ## XTicks 
        relPlot.axes[0][i].set_xticklabels(xtl)                 ## XTick Labels
        relPlot.axes[0][i].tick_params(axis='x', rotation=60);  ## XTicks Rotation
        relPlot.axes[0][i].grid(b=True, which='major', color='grey', linestyle='--',alpha=0.3)
        relPlot.axes[0][i].grid(b=True, which='minor', color='b', linestyle='--',alpha=0.1)
        relPlot.axes[0][i].set_yticks(yt)
        relPlot.axes[0][i].text(.48,.94,KDPImm[i],horizontalalignment='center',transform=relPlot.axes[0][i].transAxes)
        relPlot.axes[0][i].set_title("")
        relPlot.axes[0][i].spines['right'].set_visible(True)
        relPlot.axes[0][i].spines['top'].set_visible(True)

    
    relPlot.axes[0][0].set_yticks(yt)                       ## YTicks 
    relPlot.axes[0][0].set_yticklabels(ytl)                 ## YTick Labels
#     relPlot.legend(bbox_to_anchor= (1.03, 1) )
#     relPlot.fig.legend(handles=HANDLES, labels=LABELS, loc='upper center',fancybox=True,ncol=3,bbox_to_anchor=(.5,1.05))
    
    l = Age
    relPlot.fig.legend(h, l, loc='upper center',title='',ncol=3,bbox_to_anchor=(0.5,1.05), frameon=False)
    
    plt.tight_layout();
    name = "Relplot_" + plotExtension + "2.png"
    plt.tight_layout()
    plt.savefig(name,dpi=300,bbox_inches='tight');
    plt.show()
    pass;

agewise_Relplot(df_nucleation_melt_Age,"AGE_Vaccination_1","Vaccine_PresentRate",True)
# agewise_Relplot(df_nucleation_melt_Age,"AGE_Vaccination_1_5","Vaccine_BestWeekAvg",True)
# agewise_Relplot(df_nucleation_melt_Age,"AGE_Vaccination_2","Vaccine_TwicePresent",True)

In [ ]:
df_nucleation_melt_Age

In [ ]:
######Compute - Ratio Table-----------#######
CIR=np.zeros(2)
# Denom2=486
# Denom3=324
# KDPwsat1=np.zeros(3)
# KDPwsat2=np.zeros(3)
# KDPg1=np.zeros(3)
# KDPg2=np.zeros(3)
# KDPcp=np.zeros(3)
# KDPlambda=np.zeros(2)



#list_param=["CIR40","CIR50","0_33", "0_66" ,"1_00","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
#list_head=["CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat1","KDPwsat2","KDPwsat2","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
list_param=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head=["KDPwsat2","KDPwsat2","KDPwsat2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]


#list_param=["CIR40","CIR50","0_33", "0_66" ,"1_00","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
#list_head=["CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat1","KDPwsat2","KDPwsat2","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
lenArray = len(list_param)
table=np.zeros([lenArray,lenArray])
df_prob = df_active[df_active['Date Of Peak']==NOWAVEPEAKDATE].copy(deep=True);           
for i in range(len(list_param)):
    print(" List : ", list_head[i] ,"  ", list_param[i] )
    for j in range(len(list_param)):
        if(i!=j):

            df_prob1 = df_prob[df_prob[list_head[i]] == list_param[i] ].copy(deep=True)
            df_prob2 = df_prob1[df_prob1[list_head[j]] == list_param[j] ]
            val = df_prob2.shape[0]
            table[i][j] = val
#                 print(" val : ", val)
        else:
            df_prob1 = df_prob[df_prob[list_head[i]] == list_param[i] ].copy(deep=True); 
            val = df_prob1.shape[0]
            table[i][j] = val
#                 print(" val : ", val)
               
print(table.max())
# fig, ax = plt.subplots(figsize=[11,8])
# im = ax.imshow(table, cmap="RdYlGn")

# ax.set_xticks(np.arange(len(list_param)))
# ax.set_yticks(np.arange(len(list_param)))

# cbar.ax.set_ylabel("Number of No Wave 3 Scenarios", rotation=-90, va="bottom")

# # ... and label them with the respective list entries
# ax.set_xticklabels(list_param)
# ax.set_yticklabels(list_param)
# cbar = ax.figure.colorbar(im, ax=ax)
# cbar.ax.set_ylabel("Number of scenarios with no Wave 3", rotation=-90, va="bottom")

# # Rotate the tick labels and set their alignment.
# plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
#          rotation_mode="anchor")
# plt.savefig("heatmap_wave3.png")
# plt.show()

In [ ]:
table1=np.zeros([lenArray,lenArray])

for i in range(len(list_param)):
    for j in range(len(list_param)):
        if(i==j):
            if(i==0 or i==1 or i==11 or i==12):
                table1[i][j]=table[i][j]/486
            else:
                table1[i][j]=table[i][j]/324
        else:
            if(i==0 or i==1 or i==11 or i==12 or j==0 or j==1 or j==11 or j==12):
                if(i==0 and j!=11 and j!=12):
                    table1[i][j]=table[i][j]/162
                elif(i==1 and j!=11 and j!=12):
                    table1[i][j]=table[i][j]/162
                elif(j==0 and i!=11 and i!=12 ):
                    table1[i][j]=table[i][j]/162
                elif(j==1 and i!=11 and i!=12):
                    table1[i][j]=table[i][j]/162
                else:
                    table1[i][j]=table[i][j]/243
            else:
                table1[i][j]=table[i][j]/108

          

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
  
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=False, bottom=True,
                   labeltop=False, labelbottom=True)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)
    


    return im, cbar




In [ ]:
import matplotlib
def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
   

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
#     if threshold is not None:
#         threshold = im.norm(threshold)
#     else:
#         threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            #kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, (data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
fig, ax = plt.subplots(figsize=[11,8])
color_map=plt.cm.get_cmap("summer")
reversed_cmap=color_map.reversed()
list_label=["CIR40","CIR50","IENV-33P", "IENV-66P" ,"IENV-100P","IENV-Jul21" ,"IENV-Sep21", "IENV-Nov21","ABW-33P", "ABW-66P" ,"ABW-100P","ABW-150" ,"ABW-180","VR-100P", "VR-150P", "VR-200P","SD-Good","SD-Bad","SD-Worse"]
im, cbar = heatmap(table1, list_label, list_label, ax=ax,
                   cmap=reversed_cmap, cbarlabel="Ratio of No Wave 3 Scenarios")

#texts = annotate_heatmap(im, valfmt="{x:.1f} t")
# textstr = '\n'.join((
# "CIR: Case to infection ratio",
# "IENV: Immune-escape new variant",
# "ABW: Antibody waning","VR: Vaccine rate","SD: Social distancing"))

# props = dict(boxstyle='round', facecolor='white', alpha=1.0, edgecolor='black')

# # place a text box in upper left in axes coords
# ax.text(-0.6, -0.25, textstr, transform=ax.transAxes, fontsize=14,
#         verticalalignment='top', bbox=props)
fig.tight_layout()
plt.savefig("heatmap_nowave3.png",dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
table2=np.zeros([lenArray,lenArray])
for i in range(len(list_param)):
    for j in range(len(list_param)):
        table2[i][j]=1-table1[i][j]

In [ ]:
fig, ax = plt.subplots(figsize=[11,8])
color_map=plt.cm.get_cmap("hot")
reversed_cmap=color_map.reversed()
#list_label=["CIR:40","CIR:50","IENV Intensity:33%", "IENV Intesnity:66%" ,"IENV Intensity:100%","IENV Jul'21" ,"IENV Sep'21", "IENV Nov'21","ABW Intensity:33%", "ABW Intensity:66%" ,"ABW Intensity:100%","ABW:150 Days" ,"ABW:180 Days","VR: Present", "VR: 1.5x Present", "VR: 2x Present","SD: Normal","SD: Bad","SD: Worse"]
im, cbar = heatmap(table2, list_label, list_label, ax=ax,
                   cmap=reversed_cmap, cbarlabel="Ratio of Wave 3 Scenarios")
#texts = annotate_heatmap(im, valfmt="{x:.1f} t")
# textstr = '\n'.join((
# "CIR: Case to infection ratio",
# "IENV: Immune-escape new variant",
# "ABW: Antibody waning","VR: Vaccine rate","SD: Social distancing"))

# props = dict(boxstyle='round', facecolor='white', alpha=1.0, edgecolor='black')

# # place a text box in upper left in axes coords
# ax.text(-0.6, -0.25, textstr, transform=ax.transAxes, fontsize=14,
#         verticalalignment='top', bbox=props)
fig.tight_layout()
plt.savefig("heatmap_wave3.png",dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
import statistics
df_heat = df_active.copy(deep=True)

#df_heat = df_heat[df_heat['Date Of Peak']!=NOWAVEPEAKDATE] 

# list_param=["CIR40","CIR50","0_33", "0_66" ,"1_00","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
# list_head=["CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat1","KDPwsat2","KDPwsat2","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
list_param=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head=["KDPwsat2","KDPwsat2","KDPwsat2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]


#list_param=["CIR40","CIR50","0_33", "0_66" ,"1_00","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
#list_head=["CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat1","KDPwsat2","KDPwsat2","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
lenArray = len(list_param)
table_date=np.zeros([lenArray,lenArray])
table_prob=np.zeros([lenArray,lenArray])
#df_prob = df_active[df_active['Date Of Peak']==NOWAVEPEAKDATE].copy(deep=True);           
for i in range(len(list_param)):
    print(" List : ", list_head[i] ,"  ", list_param[i] )
    for j in range(len(list_param)):
        if(i!=j):

            df_heat1 = df_heat[df_heat[list_head[i]] == list_param[i] ].copy(deep=True)
            df_heat2 = df_heat1[df_heat1[list_head[j]] == list_param[j] ]
            denom = df_heat2.shape[0]
            df_heat2 = df_heat2[df_heat2['Date Of Peak']!=NOWAVEPEAKDATE] 
            num=df_heat2.shape[0]
            if num==0:
                table_date[i][j]=0
                table_prob[i][j]=0
            else:
                val = int(statistics.median(df_heat2['Date Of Peak']))
                table_date[i][j] = val
                table_prob[i][j] = num/denom
                
#                 print(" val : ", val)
        else:
            
            df_heat1 = df_heat[df_heat[list_head[i]] == list_param[i] ].copy(deep=True); 
            denom = df_heat1.shape[0]
            df_heat1= df_heat1[df_heat1['Date Of Peak']!=NOWAVEPEAKDATE]
            num=df_heat1.shape[0]
            if num==0:
                table_date[i][j]=0
                table_prob[i][j]=0
            else:
                val = int(statistics.median(df_heat1['Date Of Peak']))
                table_date[i][j] = val
                table_prob[i][j] = num/denom
#                 print(" val : ", val)

In [ ]:
table_prob

In [ ]:
dd=np.array(date_generated)
date_heat=np.empty(shape=(lenArray,lenArray),dtype='<U9')
for i in range(lenArray):
    for j in range(lenArray):
        date_heat[i][j]=date_generated[int(table_date[i][j])]
        #print(date_generated[int(table_date[i][j])])
fig, ax = plt.subplots(figsize=[50,40])
color_map=plt.cm.get_cmap("hot")
reversed_cmap=color_map.reversed()
list_label=["CIR:40","CIR:50","IENV Intensity:33%", "IENV Intesnity:66%" ,"IENV Intensity:100%","IENV Jul'21" ,"IENV Sep'21", "IENV Nov'21","ABW Intensity:33%", "ABW Intensity:66%" ,"ABW Intensity:100%","ABW:150 Days" ,"ABW:180 Days","VR: Present", "VR: 1.5x Present", "VR: 2x Present","SD: Normal","SD: Bad","SD: Worse"]
im, cbar = heatmap(table2, list_label, list_label, ax=ax,
                   cmap=reversed_cmap, cbarlabel="Ratio of Wave 3 Scenarios")
texts = annotate_heatmap(im,date_heat)

fig.tight_layout()
date_heat


In [3]:
date_generated

NameError: name 'date_generated' is not defined

In [ ]:
####### COMPUTE ######################
## @@ No Wave Heat Map probability ###

import statistics
def HeatPlotDataGenerator(DataFrame,list_param,list_head):
    ## list of Parameter

    ## Arrays to Store probabilitites
    noWaveProbArray = np.zeros(shape=(len(list_param),len(list_head)))
    waveProbArray = np.zeros(shape=(len(list_param),len(list_head)))
    MedianNumberAtPeak = np.zeros(shape=(len(list_param),len(list_head)))
    MeanNumberAtPeak = np.zeros(shape=(len(list_param),len(list_head)))
    # Copt of Dataframe
    df_heat_master = df_active.copy(deep=True)
    
    for i in range(len(list_param)):
        for j in range(len(list_param)):
            df_heat1 = df_heat_master[(df_heat_master[list_head[i]] == list_param[i]) & (df_heat_master[list_head[j]] == list_param[j]) ]            
            
            totalCases = df_heat1.shape[0]
            
            
            if((list_head[i] == list_head[j]) and (i != j)):
                noWaveProbArray[i][j] = 10;
                waveProbArray[i][j]   = 10;
                MedianNumberAtPeak[i][j] = 50;
                MeanNumberAtPeak[i][j]=50
            
            if(totalCases):
                ## remove no wave scenarios
                df_heat3=df_heat1.copy(deep=True)
                MeanNumberAtPeak[i][j]=math.log10(np.mean(df_heat3['Number At Peak']))
                df_heat2 = df_heat1[df_heat1["Date Of Peak"] != NOWAVEPEAKDATE]
                
                wave3Cases    = df_heat2.shape[0]
                noWave3Cases  = totalCases - wave3Cases

                ratioWave3    = wave3Cases/totalCases
                ratioNoWave3  = noWave3Cases/totalCases

                noWaveProbArray[i][j] = ratioNoWave3
                waveProbArray[i][j]   = ratioWave3
                if(wave3Cases !=0):
                    MedianNumberAtPeak[i][j] = statistics.median(df_heat2["LogNumAtPeak"])
                
            
    return MedianNumberAtPeak,noWaveProbArray,waveProbArray,MeanNumberAtPeak
    

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
  
    if not ax:
        ax = plt.gca()
    # Plot the heatmap
    im = ax.imshow(data, **kwargs)
    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    
    ax.set_facecolor("#DCDCDC")
    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=False, bottom=True,
                   labeltop=False, labelbottom=True)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")
    # Turn spines off and create white grid.
    
    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)
    

    return im, cbar



In [ ]:
###### PLOTTER SCRIPT #######
## Heat Wave Plotter ######

import matplotlib.colors
import math
def HeatwavePlotter(DataFrame,list_param,list_head,list_label,PlotExtension):
    
    numatpeak, noWaveArray,waveArray,meanatpeak = HeatPlotDataGenerator(DataFrame,list_param, list_head)
    
#     fig, ax = plt.subplots(figsize=[11,8])
#     color_map=plt.cm.get_cmap("summer")
#     reversed_cmap=color_map.reversed()
#     print(reversed_cmap)
#     im, cbar = heatmap(noWaveArray, list_label, list_label, ax=ax,
#                        cmap=reversed_cmap ,vmin=0, vmax=1)
#     cbar.set_label("conditional probability of no Wave-3",size=18)
#     fig.tight_layout()
#     ax.patch.set_facecolor('white')
#     plt.rcParams['axes.facecolor']='white'
    
#     name ="heatmap_nowave3" + PlotExtension + ".png"
#     plt.savefig(name,dpi=300,bbox_inches='tight')
#     plt.show()
    
    
    fig, ax = plt.subplots(figsize=[11,8])
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","#800000"])
    cmap.set_over('#808080')
#     color_map=plt.cm.get_cmap("autumn")
#     reversed_cmap=cmap.reversed()
    ax.patch.set_facecolor('white')
    plt.rcParams['axes.facecolor']='white'
    flat=numatpeak.flatten()
    flat.sort()
    flat= np.unique(flat)
    maxVal = flat[-2]
    
    im, cbar = heatmap(meanatpeak, list_label, list_label, ax=ax,
                       cmap=cmap, cbarlabel="Log of Mean of (Active cases at peak)",vmin=4, vmax=maxVal)
    
    

    
#     for i in range(len(list_param)):
#         for j in range(len(list_head)):
#             if(waveArray[i][j] != 10):
#                 num = np.round(waveArray[i][j],2)
#                 strnum = str(num)
#                 strnum = strnum[:4]
#                 text = ax.text(i, j, strnum,
#                                ha="center", va="center", color="w")
#                 print(np.round(waveArray[i][j],2))
#                 print(round(waveArray[i][j],2))
    fig.tight_layout()
    
    ax.patch.set_facecolor('#FFFFFF')
    name ="heatmap_wave3" + PlotExtension + ".png"
    plt.savefig(name,dpi=300,bbox_inches='tight')
    plt.show()

    


list_param=["CIR40","CIR50","0_33", "0_66" ,"1_00","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head=["CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat1","KDPwsat2","KDPwsat2","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]   
list_label=["CIR40","CIR50","IENV-33P", "IENV-66P" ,"IENV-100P","IENV-Jul21" ,"IENV-Sep21", "IENV-Nov21","ABW-33P", "ABW-66P" ,"ABW-100P","ABW-150" ,"ABW-180","VR-100P", "VR-150P", "VR200P","SD-Good","SD-Bad","SD-Worse"]


list_param=["SD_RA", "Vaccine_TwicePresent", "ImmunEscp_Nov","CIR40","CIR50","0_33", "0_66" ,"ImmunEscp_Sep","0_33", "0_66" ,"1_00","ABW150Days" ,"ABW180Days","Vaccine_PresentRate", "Vaccine_BestWeekAvg","1_00","ImmunEscp_Jul" ,"SD_RAR","SD_RARF"]
list_head=["SD","Vaccine Rate","KDPwsat2","CIR","CIR","KDPwsat1","KDPwsat1","KDPwsat2","KDPg1","KDPg1","KDPg1","KDPg2","KDPg2","Vaccine Rate","Vaccine Rate","KDPwsat1","KDPwsat2","SD","SD"]   
list_label=["SD-Good", "IENV-Nov21", "VR200P","CIR40","CIR50","IENV-33P", "IENV-66P" ,"IENV-Sep21","ABW-33P", "ABW-66P" ,"ABW-100P","ABW-150" ,"ABW-180","VR-100P", "VR-150P" ,"IENV-100P","IENV-Jul21","SD-Bad","SD-Worse"]

list_param=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head=["KDPwsat2","KDPwsat2","KDPwsat2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
list_label=["IENV-Jul21" ,"IENV-Sep21", "IENV-Nov21","VR-100P", "VR-150P", "VR-200P","SD-Good","SD-Bad","SD-Worse"]

HeatwavePlotter(df_active,list_param,list_head,list_label,"Short_Numatpeak")


In [1]:
##### TABLE GENERATOR ######
## @@ Generate Probability Table ##

import statistics
import scipy.stats as st


def TableGenerator(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param):
    ## list of Parameter

    ## Arrays to Store probabilitites
    waveProbArray             = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    MeanNumberAtPeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    CIMeanNumberAtPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    stddevMeanNumberAtPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr1MeanNumberAtPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr2MeanNumberAtPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    MedianNumberAtPeak        = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    CIMedianNumberAtPeak        = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    stddevMedianNumberAtPeak        = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr1MedianNumberAtPeak        = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr2MedianNumberAtPeak        = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    MeanDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)),dtype='object')
    CIMeanDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    stddevMeanDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr1MeanDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr2MeanDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    MedianDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)),dtype='object')
    CIMedianDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    stddevMedianDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr1MedianDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqr2MedianDateAtpeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    DateAtPeak40p=np.zeros(shape=(len(Row_Param),len(Col_Param1)),dtype='object')
    DateAtPeak60p=np.zeros(shape=(len(Row_Param),len(Col_Param1)),dtype='object')
    MinNumAtPeak=np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    MaxNumAtPeak=np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    

    # Copt of Dataframe
    df_heat_master = df_active.copy(deep=True)
    
    for i in range(len(Row_Param)):
        
        ## Filter for Both Rows and SubRows
        df_heat_Row =  df_heat_master[df_heat_master["SD"] == Row_Param[i]].copy(deep=True)
        
        for j in range(len(Col_Param1)):
            
            ## Filter for j - First Parameeter - Vaccine Rate
            df_heat_col1 = df_heat_Row[df_heat_Row["Vaccine Rate"] == Col_Param1[j]]
            df_heat1     = df_heat_col1[df_heat_col1["KDPwsat2"] == Col_Param2[j]]
            
            totalCases = df_heat1.shape[0]
            
            ## remove no wave scenarios
            df_heat2 = df_heat1[df_heat1["Date Of Peak"] != NOWAVEPEAKDATE]
            wave3Cases    = df_heat2.shape[0]
            if(wave3Cases):

                ratioWave3    = wave3Cases/totalCases
                waveProbArray[i][j] = ratioWave3

                #### MEDIAN NUMBER AT PEAK  
#                 MedianNumberAtPeak[i][j] = int(statistics.median(df_heat2['Number At Peak']))
# #                 ci_MednumatPeak=st.t.interval(0.95,wave3Cases-1,loc=np.median(df_heat2['Number At Peak']),scale=scipy.stats.sem(df_heat2['Number At Peak']))
# #                 CIMedianNumberAtPeak[i][j] = int(abs(  max(ci_MednumatPeak)  - MedianNumberAtPeak[i][j] ))
# #                 stddev_MednumatPeak=st.t.std(wave3Cases-1,loc=np.median(df_heat2['Number At Peak']),scale=scipy.stats.sem(df_heat2['Number At Peak']))
# #                 stddevMedianNumberAtPeak[i][j] = int(stddev_MednumatPeak)
#                 iqr_MednumatPeak=st.iqr(df_heat2['Number At Peak'])
#                 iqr1MedianNumberAtPeak[i][j] = int(iqr_MednumatPeak)
                
                
#                 #### MEAN NUMBER AT PEAK 
#                 MeanNumberAtPeak[i][j] = int(statistics.mean(df_heat2['Number At Peak']))
# #                 ci_MeannumatPeak=st.t.interval(0.95,wave3Cases-1,loc=np.mean(df_heat2['Number At Peak']),scale=scipy.stats.sem(df_heat2['Number At Peak']))
# #                 CIMeanNumberAtPeak[i][j] = int(abs(  max(ci_MeannumatPeak)  - MeanNumberAtPeak[i][j] ))
# #                 stddev_MeannumatPeak=st.t.std(wave3Cases-1,loc=np.mean(df_heat2['Number At Peak']),scale=scipy.stats.sem(df_heat2['Number At Peak']))
# #                 stddevMeanNumberAtPeak[i][j] = int(stddev_MeannumatPeak)
#                 iqr_MeannumatPeak=st.iqr(df_heat2['Number At Peak'])
#                 iqr1MeanNumberAtPeak[i][j] = int(iqr_MeannumatPeak)
                
                
                lowerp=25
                upperp=75
                #### MEDIDAN DATE OF PEAK
                #meanDate = int(statistics.median(df_heat2['Date Of Peak']))
                MedianDateAtpeak[i][j] = date_generated[int(np.percentile(df_heat2['Date Of Peak'],50,interpolation='lower'))]
#                 ci_Med_DOP=st.t.interval(0.95,wave3Cases-1,loc=np.median(df_heat2['Date Of Peak']),scale=scipy.stats.sem(df_heat2['Date Of Peak']))
#                 CIMedianDateAtpeak[i][j] = int(abs(  max(ci_Med_DOP) - meanDate ))
#                 iqr_Med_DOP=st.t.std(wave3Cases-1,loc=np.median(df_heat2['Date Of Peak']),scale=scipy.stats.sem(df_heat2['Date Of Peak']))
#                 iqr1MedianDateAtpeak[i][j] = int(iqr_Med_DOP)t
                DateAtPeak40p[i][j]=date_generated[int(np.percentile(df_heat2['Date Of Peak'],lowerp,interpolation='lower'))]
                DateAtPeak60p[i][j]=date_generated[int(np.percentile(df_heat2['Date Of Peak'],upperp,interpolation='lower'))]
                df_heat3=df_heat2.copy(deep=True)
                df_heat4=df_heat2.copy(deep=True)
                df_heat3=df_heat3[df_heat3['Date Of Peak']==int(np.percentile(df_heat2['Date Of Peak'],50,interpolation='lower'))]
                
                df_heat4 = df_heat4[df_heat4['Date Of Peak']>=np.percentile(df_heat2['Date Of Peak'],lowerp,interpolation='lower')]
                df_heat4 = df_heat4[df_heat4['Date Of Peak']<=np.percentile(df_heat2['Date Of Peak'],upperp,interpolation='lower')]
               
                MinNumAtPeak[i][j]=np.min(df_heat4['Number At Peak'])
                MaxNumAtPeak[i][j]=np.max(df_heat4['Number At Peak'])
                MeanNumberAtPeak[i][j]=int(np.mean(df_heat3['Number At Peak']))
                
    
                
                
                
#                 #### MEan Date Of Peak
#                 medianDate = int(statistics.median(df_heat2['Date Of Peak']))
#                 MeanDateAtpeak[i][j]  =  date_generated[int(statistics.mean(df_heat2['Date Of Peak']))]
# #                 ci_Mean_DOP=st.t.interval(0.95,wave3Cases-1,loc=np.mean(df_heat2['Date Of Peak']),scale=scipy.stats.sem(df_heat2['Date Of Peak']))
# #                 CIMeanDateAtpeak[i][j] = int(abs(  max(ci_Mean_DOP) - medianDate ))
#                 iqr_Mean_DOP=st.t.std(wave3Cases-1,loc=np.mean(df_heat2['Date Of Peak']),scale=scipy.stats.sem(df_heat2['Date Of Peak']))
#                 iqr1MeanDateAtpeak[i][j] = int(iqr_Mean_DOP)
                
                
                
            
            else:
                MedianDateAtpeak[i][j] = "No Wave 3"
                

            
#     return waveProbArray, MedianNumberAtPeak, CIMedianNumberAtPeak,MedianNumberAtPeak,CIMedianNumberAtPeak, \
#            MedianDateAtpeak,CIMedianDateAtpeak,MeanDateAtpeak,CIMeanDateAtpeak
#     return waveProbArray, MedianNumberAtPeak, stddevMedianNumberAtPeak,MeanNumberAtPeak,stddevMeanNumberAtPeak, \
#            MedianDateAtpeak,stddevMedianDateAtpeak,MeanDateAtpeak,stddevMeanDateAtpeak,iqr1MedianNumberAtPeak,iqr2MedianNumberAtPeak,\
#            iqr1MeanNumberAtPeak,iqr2MeanNumberAtPeak,iqr1MedianDateAtpeak,iqr2MedianDateAtpeak,iqr1MeanDateAtpeak,iqr2MeanDateAtpeak
    return MedianDateAtpeak,DateAtPeak40p,DateAtPeak60p,MeanNumberAtPeak,MinNumAtPeak,MaxNumAtPeak

list_param1=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head1=["KDPwsat2","KDPwsat2","KDPwsat2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
list_label=["IENV-Jul21" ,"IENV-Sep21", "IENV-Nov21","VR-100P", "VR-150P", "VR-200P","SD-Good","SD-Bad","SD-Worse"]

Col_Param2  = ["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov"]
Col_Param1  = ["Vaccine_PresentRate", "Vaccine_PresentRate", "Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_BestWeekAvg", "Vaccine_BestWeekAvg","Vaccine_TwicePresent","Vaccine_TwicePresent","Vaccine_TwicePresent", ]
ColHead     = ["KDPwsat2","Vaccine Rate"]

RowHead     = ["SD"]
Row_Param   = ["SD_RA","SD_RAR","SD_RARF"]

# waveProbArray, MedianNumberAtPeak, CIMedianNumberAtPeak,MedianNumberAtPeak,CIMedianNumberAtPeak,MedianDateAtpeak,CIMedianDateAtpeak,MeanDateAtpeak,CIMeanDateAtpeak  = TableGenerator(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param)
# waveProbArray, MedianNumberAtPeak, stddevMedianNumberAtPeak,MeanNumberAtPeak,stddevMeanNumberAtPeak,MedianDateAtpeak,stddevMedianDateAtpeak,MeanDateAtpeak,stddevMeanDateAtpeak,iqr1MedianNumberAtPeak,iqr2MedianNumberAtPeak,\
#            iqr1MeanNumberAtPeak,iqr2MeanNumberAtPeak,iqr1MedianDateAtpeak,iqr2MedianDateAtpeak,iqr1MeanDateAtpeak,iqr2MeanDateAtpeak = TableGenerator(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param)
MedianDateAtpeak,DateAtPeak40p,DateAtPeak60p,MeanNumberAtPeak,MinNumAtPeak,MaxNumAtPeak= TableGenerator(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param)

NameError: name 'df_active' is not defined

In [ ]:
# np.set_printoptions(linewidth=200)

def formatter(array):
    ans =  np.zeros(shape=(array.shape[0],array.shape[1]), dtype = object)
    row = array.shape[0]
    col = array.shape[1]
    
    for i in range(row):
        for j in range(col):
            val = array[i][j]
            if(val == 0):
                ans[i][j] = "0"
            elif( int(val / 1000000) >= 1):
                ans[i][j] =  str(int(val/1000000)) + "M"
#                 print (str(int(val/1000000)) + "M")
            elif( int(val / 1000) >= 1):
                ans[i][j] =  str(int(val/1000)) + "K"
#                 print (str(int(val/1000)) + "K")
            else:
                ans[i][j] =  str(int(val))
            
            
        pass;
    return ans;
    pass;

ans  = formatter(MaxNumAtPeak)
ans = DateAtPeak60p

for i in range(ans.shape[0]):
    for j in range(ans.shape[1]):
        print(ans[i][j] ,",",end = '')
    print("\n",end = '')


In [2]:
##### TABLE GENERATOR ######
## @@ AGEWISE Cases Probability Table ##

import statistics
import scipy.stats as st


def TableGenerator_Age(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param):
    ## list of Parameter

    ## Arrays to Store probabilitites
    waveProbArray             = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    MeanRatioPeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    CIMeanRatioPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    stddevMeanRatioPeak=np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqrMeanRatioPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    minRatioPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    maxRatioPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    RatioPeaklowerp=np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    RatioPeakupperp=np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    lowerp=25
    upperp=75
    
    MedianRatioPeak          = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    CIMeanRatioPeakLower  = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    CIMeanRatioPeakUpper  = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    iqrMedianRatioPeak   = np.zeros(shape=(len(Row_Param),len(Col_Param1)))
    
    KDP_lambda_name=["SD_RA","SD_RAR","SD_RARF"]
    

    
    for i in range(len(Row_Param)):
        
        ## Filter for Both Rows and SubRows
        df_heat_Row_m =  Row_Param[i].copy(deep=True)
        
        df_heat_Row = df_heat_Row_m[df_heat_Row_m["SD"] == KDP_lambda_name[i%3]]
#         df_heat_Row=  Row_Param[i].copy(deep=True)
        
        for j in range(len(Col_Param1)):
            
            ## Filter for j - First Parameeter - Vaccine Rate
            df_heat_col1 = df_heat_Row[df_heat_Row["Vaccine Rate"] == Col_Param1[j]]
            df_heat1     = df_heat_col1[df_heat_col1["KDPwsat2"] == Col_Param2[j]]
            
            totalCases = df_heat1.shape[0]
            
            ## remove no wave scenarios
            df_heat2 = df_heat1[df_heat1["Date Of Peak"] != NOWAVEPEAKDATE].copy(deep=True)
            wave3Cases    = df_heat2.shape[0]
            if(wave3Cases):

                ratioWave3    = wave3Cases/totalCases
                waveProbArray[i][j] = ratioWave3
                
                wave2List = df_heat2[324]
                Wave3List = df_heat2['Number At Peak'].copy(deep = True)
                RatioList = Wave3List.div(wave2List)
                
                
                #### MEDIAN NUMBER AT PEAK  
                MedianRatioPeak[i][j] = np.median(RatioList)
                RatioPeaklowerp[i][j]=np.percentile(RatioList,lowerp)
                RatioPeakupperp[i][j]=np.percentile(RatioList,upperp)
#                 ci_MedianRatioAtPeak=st.t.interval(0.95,wave3Cases-1,loc=np.median(RatioList),scale=scipy.stats.sem(RatioList))
                iqr_MedianRatioAtPeak=st.iqr(RatioList)
                #print(ci_MedianRatioAtPeak)
                #CIMedianRatioPeak[i][j] = abs(  max(ci_MedianRatioAtPeak)  - MedianRatioPeak[i][j] )
                iqrMedianRatioPeak[i][j]=iqr_MedianRatioAtPeak
                min_MedianRatioAtPeak=np.min(RatioList)
                minRatioPeak[i][j]=np.min(RatioList)
                maxRatioPeak[i][j]=np.max(RatioList)
                
                
                ### MEan Ratio at peak
                MeanRatioPeak[i][j] = statistics.mean(RatioList)
                ci_MeanRatioAtPeak=st.t.interval(0.95,wave3Cases-1,loc=np.mean(RatioList),scale=scipy.stats.sem(RatioList))
                iqr_MeanRatioAtPeak=st.iqr(RatioList)
                #print(ci_MedianRatioAtPeak)
                CIMeanRatioPeak[i][j] = abs(  max(ci_MeanRatioAtPeak)  - MeanRatioPeak[i][j] )
                CIMeanRatioPeakLower[i][j]=ci_MeanRatioAtPeak[0]
                CIMeanRatioPeakUpper[i][j]=ci_MeanRatioAtPeak[1]
                iqrMeanRatioPeak[i][j] = iqr_MeanRatioAtPeak
                stddevMeanRatioPeak[i][j]=np.std(RatioList)
                
                            
#             else:
#                 MedianDateAtpeak[i][j] = "No Wave 3"
                

#                 plt.hist(RatioList)
#                 plt.title(str(KDP_lambda_name[i%3])+" "+str(Col_Param1[j])+" "+str(Col_Param2[j]))
#                 plt.show()
    return MedianRatioPeak,RatioPeaklowerp,RatioPeakupperp , RatioList,CIMeanRatioPeak,stddevMeanRatioPeak,CIMeanRatioPeakLower,CIMeanRatioPeakUpper,MeanRatioPeak

list_param1=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent","SD_RA","SD_RAR","SD_RARF"]
list_head1=["KDPwsat2","KDPwsat2","KDPwsat2","Vaccine Rate","Vaccine Rate","Vaccine Rate","SD","SD","SD"]
list_label=["IENV-Jul21" ,"IENV-Sep21", "IENV-Nov21","VR-100P", "VR-150P", "VR-200P","SD-Good","SD-Bad","SD-Worse"]

Col_Param2  = ["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov","ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov"]
Col_Param1  = ["Vaccine_PresentRate", "Vaccine_PresentRate", "Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_BestWeekAvg", "Vaccine_BestWeekAvg","Vaccine_TwicePresent","Vaccine_TwicePresent","Vaccine_TwicePresent", ]
ColHead     = ["KDPwsat2","Vaccine Rate"]

RowHead     = ["SD"]
Row_Param   = [df_nucleation_11,df_nucleation_11,df_nucleation_11,
               df_nucleation_18,df_nucleation_18,df_nucleation_18,
               df_nucleation_45,df_nucleation_45,df_nucleation_45,
               df_nucleation_60,df_nucleation_60,df_nucleation_60,
              df_nucleation_60plus,df_nucleation_60plus,df_nucleation_60plus]
# Row_Param   = [df_nucleation_11,
#                df_nucleation_18,
#                df_nucleation_45,
#                df_nucleation_60,
#               df_nucleation_60plus]
# minRatioPeak,maxRatioPeak,waveProbArray, MedianRatioPeak, CIMedianRatioPeak , MeanRatioPeak , CIMeanRatioPeak,iqrMeanRatioPeak,iqrMedianRatioPeak, RatioList= \
# TableGenerator_Age(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param)
MedianRatioPeak,RatioPeaklowerp,RatioPeakupperp, RatioList,CIMeanRatioPeak,stddevMeanRatioPeak,CIMeanRatioPeakLower,CIMeanRatioPeakUpper,MeanRatioPeak=TableGenerator_Age(df_active,Col_Param1,Col_Param2,ColHead,RowHead,Row_Param)

NameError: name 'df_nucleation_11' is not defined

In [ ]:
# np.set_printoptions(linewidth=200)

def formatter(array):
    ans =  np.zeros(shape=(array.shape[0],array.shape[1]), dtype = object)
    row = array.shape[0]
    col = array.shape[1]
    
    for i in range(row):
        for j in range(col):
            val = array[i][j]
            if(val == 0):
                ans[i][j] = "0"
            elif( int(val / 1000000) >= 1):
                ans[i][j] =  str(int(val/1000000)) + "M"
#                 print (str(int(val/1000000)) + "M")
            elif( int(val / 1000) >= 1):
                ans[i][j] =  str(int(val/1000)) + "K"
#                 print (str(int(val/1000)) + "K")
            else:
                ans[i][j] =  str(int(val))
            
            
        pass;
    return ans;
    pass;

# ans  = formatter(MaxNumAtPeak)
# ans = MedianRatioPeak
ans = CIMeanRatioPeakUpper
for i in range(ans.shape[0]):
    for j in range(ans.shape[1]):
        print("{:.2f}".format(ans[i][j]) ,",",end = '')
    print("\n",end = '')


In [ ]:
### ---- FUNCTION DEFINITION  ----   ####
### @@ DataGenerator - Immune Escape ratio   ###



# DATA GENERATOR - NUMBER OF CASES
####################################_____[[ DATA GENERERATOR ]] FOR NUMBER OF CASES ______________________ #################################

from scipy.stats import norm
import numpy
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.utils.fixes import parse_version
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt



def GenerateData(df_active,YColumn,Bandwidth,N_bins,LinePLotVals_Cases,LinePLotX_Cases,NoWaveRatio,boolHist):
    fig, ax = plt.subplots(3,2,figsize=(11,8))
    colorVal = ['#7aa6c2','#4acf94','#b99bce']
    kernel = 'epanechnikov'
    KDPCIR=["CIR= 40", "CIR= 50"]
    KDPImm=["EEscape Period - July","Escape Period - September", "Escape Period  - November"]
    legend = ["Present Rate","1.5x Present Rate","2x Present Rate"]
    
    
    
    
    for i in range(3):
        for j in range(3):
            df_kde=df_active.copy(deep=True)
            df_kde=df_kde[df_kde['CIR']==KDP_nu_name[j]]
            df_kde=df_kde[df_kde['KDPwsat2']==KDP_w_sat_2_name[i]]
            
            #index for storing Lineplot arrays
            plotIndex = 0
            
            for k in range(3):
                df_kdep = df_kde.copy(deep=True)
                df_kdep = df_kdep[df_kdep['Vaccine Rate']==KDP_c_p_name[k]].copy(deep=True)
                
                TotalRows = df_kdep.shape[0]   ### NUmber of Rows for this Selected Scenario
                
                ### Remove Rows With DATE_OF_PEAK = 670
                df_kdep = df_kdep[df_kdep['Date Of Peak']!=NOWAVEPEAKDATE].copy(deep=True)
                
                ### Remove the rows with NUmber at Peak < 500 
                df_kdep = df_kdep[df_kdep['Number At Peak']>200].copy(deep=True)
                
                print(df_active.shape[0])
#                 print("Scenario : ",KDPCIR[j],"_",KDPImm[i],"_",legend[k])
#                 print("Total Rows : ", TotalRows)
#                 print("No Wave Peeks : ", df_kdep.shape[0])
#                 print("Max Peak : ", max(df_kdep['Number At Peak']) )
                ## Truncated Rows
                TruncatedRows = df_kdep.shape[0]
                
                
                hist = sns.histplot(ax=ax[i,j],data=df_kdep,x=YColumn,stat='probability',\
                                    kde=True,label=legend[k],kde_kws={"bw_adjust":Bandwidth},
                                    color=colorVal[k],bins=N_bins)
                
 
                ratio = TruncatedRows/TotalRows    #### Compute Ratio of Wave Scenarios
                
                ratioNoWave = 1.0 - ratio;
                print("Total : " ,TotalRows, " TruncatedRows : ",TruncatedRows,   "  RatioNoWave : " , ratioNoWave)
                
                               
                if(len(hist.get_lines()) == plotIndex+1):
                    ll = hist.get_lines()[k].get_data()
                    LinePLotVals_Cases[i][j][k] = np.asarray(ll[1])
                    LinePLotX_Cases[i][j][k] = np.asarray(ll[0])

                    LinePLotVals_Cases[i][j][k] *=  ratio
                    NoWaveRatio[i][j][k]         =  ratioNoWave
                    
                    plotIndex += 1
                
                else:     ##### No Cases Available
                    print("SOMETHING WRONG")
                    NoWaveRatio[i][j][k]        =   ratioNoWave
                    LinePLotVals_Cases[i][j][k] =   np.zeros(len(LinePLotVals_Cases[i][j][k]))
                    LinePLotX_Cases[i][j][k]    =   np.asarray(hist.get_lines()[0].get_data()[0])
                
#                 ###GET MAX Y VALUE for Y lim
 #                maxYVal = max(LinePLotVals_Cases[i][j][k])
                
                if(not boolHist):
                    ax[i,j].containers[0].remove() # remove the bars
                else:
                    ax[i,j].legend(loc='upper right')
                    handles, labels = ax[i,j].get_legend_handles_labels()
                
                
                ax[i,j].set_title(KDPImm[i]+", "+ KDPCIR[j])
                ax[i,j].set_xlabel(YColumn)
    #             ax[i,j].set_xlim([1,5e6])
#                ax[i,j].set_ylim([0,maxYVal+0.05])
        
                
                

#     plt.legend(handles, labels, loc='lower center',title='Vaccination Rate',fancybox=True,ncol=3,bbox_to_anchor=(0.5,0.97))
    plt.tight_layout()

    plt.show()




In [ ]:
df_active